In [3]:
import csv
import re
import time
from time import sleep
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions
from selenium.common import exceptions

import requests
headers={'User-Agent': 'Mozilla/5.0 (Linux; Android 6.0; Nexus 5 Build/MRA58N) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.96 Mobile Safari/537.36'}
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from webdriver_manager.chrome import ChromeDriverManager
import webdrivermanager


def create_webdriver_instance():
    driver = webdriver.Chrome(ChromeDriverManager().install())
    return driver


def login_to_twitter(username, password, driver):
    url = 'https://twitter.com/login'
    try:
        
        driver.get(url)
        xpath_username = '//input[@name="session[username_or_email]"]'
        WebDriverWait(driver, 10).until(expected_conditions.presence_of_element_located((By.XPATH, xpath_username)))
        uid_input = driver.find_element_by_xpath(xpath_username)
        uid_input.send_keys(username)
        time.sleep(5)
    except exceptions.TimeoutException:
        print("Timeout while waiting for Login screen")
        return False
    
    time.sleep(5)
    pwd_input = driver.find_element_by_xpath('//input[@name="session[password]"]')
    pwd_input.send_keys(password)
    try:
        pwd_input.send_keys(Keys.RETURN)
        url = "https://twitter.com/home"
        WebDriverWait(driver, 10).until(expected_conditions.url_to_be(url))
    except exceptions.TimeoutException:
        print("Timeout while waiting for home screen")
    return True


def generate_tweet_id(tweet):
    return ''.join(tweet)


def scroll_down_page(driver, last_position, num_seconds_to_load=60, scroll_attempt=0, max_attempts=60):
    """The function will try to scroll down the page and will check the current
    and last positions as an indicator. If the current and last positions are the same after `max_attempts`
    the assumption is that the end of the scroll region has been reached and the `end_of_scroll_region`
    flag will be returned as `True`"""
    end_of_scroll_region = False
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    sleep(num_seconds_to_load)
    curr_position = driver.execute_script("return window.pageYOffset;")
    if curr_position == last_position:
        if scroll_attempt < max_attempts:
            end_of_scroll_region = True
        else:
            scroll_down_page(last_position, curr_position, scroll_attempt + 1)
    last_position = curr_position
    return last_position, end_of_scroll_region


def save_tweet_data_to_csv(records, filepath, mode='a+'):
    header = ['PostDate','Description', 'image_url','source_url']
    with open(filepath, mode=mode, newline='', encoding='utf-8') as f:
        writer = csv.writer(f)
        if mode == 'w':
            writer.writerow(header)
        if records:
            writer.writerow(records)


def collect_all_tweets_from_current_view(driver, lookback_limit=30):
    """The page is continously loaded, so as you scroll down the number of tweets returned by this function will
     continue to grow. To limit the risk of 're-processing' the same tweet over and over again, you can set the
     `lookback_limit` to only process the last `x` number of tweets extracted from the page in each iteration.
     You may need to play around with this number to get something that works for you. I've set the default
     based on my computer settings and internet speed, etc..."""
    page_cards = driver.find_elements_by_xpath('//div[@data-testid="tweet"]')
    if len(page_cards) <= lookback_limit:
        return page_cards
    else:
        return page_cards[-lookback_limit:]


def extract_data_from_current_tweet_card(card):
    try:
        src_url=card.find_element_by_class_name("css-4rbku5.css-18t94o4.css-901oao.r-9ilb82.r-1loqt21.r-1q142lx.r-1qd0xha.r-a023e6.r-16dba41.r-rjixqe.r-bcqeeo.r-3s2u2q.r-qvutc0")
        print(src_url.text)
        #src_url= card.find('a',{"class":"css-4rbku5.css-18t94o4.css-901oao.r-9ilb82.r-1loqt21.r-1q142lx.r-1qd0xha.r-a023e6.r-16dba41.r-rjixqe.r-bcqeeo.r-3s2u2q.r-qvutc0"})
        #for i in src_url:
        url=src_url.get_attribute("href")
    except exceptions.NoSuchElementException:
        url=""
    try:
        image=card.find_elements_by_class_name("css-9pa8cd")

        for i in image:
            res=i.get_attribute("src")
            print(res)
            if "media" in res:
                img_src=res
            else:
                img_src=""
    except exceptions.NoSuchElementException:
        img_src = ""

    try:
        """
        If there is no post date here, there it is usually sponsored content, or some
        other form of content where post dates do not apply. You can set a default value
        for the postdate on Exception if you which to keep this record. By default I am
        excluding these.
        """
        postdate = card.find_element_by_xpath('.//time').get_attribute('datetime')
    except exceptions.NoSuchElementException:
        return
    try:
        _comment = card.find_element_by_xpath('.//div[2]/div[2]/div[1]').text
    except exceptions.NoSuchElementException:
        _comment = ""
    try:
        _responding = card.find_element_by_xpath('.//div[2]/div[2]/div[2]').text
    except exceptions.NoSuchElementException:
        _responding = ""
    tweet_text = card.text
    
    tweet = (postdate,tweet_text, img_src,url)
    print(tweet_text)
    return tweet


def main(username, password, search_term, filepath):
    save_tweet_data_to_csv(None, filepath, 'w')  # create file for saving records
    last_position = None
    end_of_scroll_region = False
    unique_tweets = set()

    driver = create_webdriver_instance()
    logged_in = login_to_twitter(username, password, driver)
    if not logged_in:
        return

    driver.get("https://twitter.com/search?q=lang%3Aen%20%26%20zoom.us%2Fmeeting&src=typed_query&f=live")
    #driver.get("https://twitter.com/search?f=live&q=lang%3Aen&src=typed_query")
#     latest=driver.find_element_by_class_name("css-4rbku5 css-18t94o4 css-1dbjc4n r-1awozwy r-oucylx r-rull8r r-wgabs5 r-1loqt21 r-6koalj r-eqz5dr r-16y2uox r-1777fci r-1ny4l3l r-xyw6el r-o7ynqc r-6416eg")
#     latest[0].text
#     search_found = find_search_input_and_enter_criteria(search_term, driver)
#     if not search_found:
#         return


    while not end_of_scroll_region:
        cards = collect_all_tweets_from_current_view(driver)
        for card in cards:
            try:
                tweet = extract_data_from_current_tweet_card(card)
            except exceptions.StaleElementReferenceException:
                continue
            if not tweet:
                continue
            tweet_id = generate_tweet_id(tweet)
            if tweet_id not in unique_tweets:
                unique_tweets.add(tweet_id)
                save_tweet_data_to_csv(tweet, filepath)
        last_position, end_of_scroll_region = scroll_down_page(driver, last_position)
        
    driver.quit()

if __name__ == '__main__':
    usr = "divyajoshi400@gmail.com"
    pwd = "moreyeahs@12345"
    path = 'Twitter_apr22.csv'
    term = 'zoom.us'

    main(usr, pwd, term, path)

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 89.0.4389
[WDM] - Get LATEST driver version for 89.0.4389


[WDM] - Driver [C:\Users\Admin\.wdm\drivers\chromedriver\win32\89.0.4389.23\chromedriver.exe] found in cache


10s
https://pbs.twimg.com/profile_images/1229859027959664641/qIFaeiGO_bigger.jpg
Cheche
@WinnieCheche
·
10s
Replying to 
@Carolinekibii
 @EnviroWild
 and 17 others
Register in advance for this meeting:
Welcome! You are invited to join a meeting: Reflection: Earth Restoration in the phase of Covid-19....
Hello, This Earth Day 2021, Join youth with restoration projects around Kenya on a discussion on how Covid-19 has affected their activities and how they've manoeuvred despite the challenges THEME:...
us02web.zoom.us
1
1
2m
https://pbs.twimg.com/profile_images/1358714323850448898/tkJ_yzmE_bigger.jpg
https://pbs.twimg.com/media/EzjyvydXEAATEhM?format=jpg&name=small
SITE_Africa
@site_africa
·
2m
It's Earth Day and we are talking about sustainable tourism development at 11:00. Will you be joining us?
https://us02web.zoom.us/meeting/register/tZYlfu6urTIuEtdPrO4AY3Em8xP2wGjddekF… 

#SITEprofs #eventprofs #meetingprofs #incentiveprofs #destinationprofs #siteunite #siteafrica #incentivetravelex

EBRD Gender
@EBRDgender
·
1h
TODAY
 
22 April, 14.00 BST

Register now for our fantastic launch event and webinar on how to advance the economic inclusion of #peoplewithdisabilities and #olderworkers

https://us02web.zoom.us/meeting/register/tZwpduCtrDMqG9MndPsnA6UKijfgPCzxVEhp…
1
1h
https://pbs.twimg.com/profile_images/1375365989177655296/bBLIdh_X_bigger.jpg
https://abs-0.twimg.com/emoji/v2/svg/1f38a.svg
https://abs-0.twimg.com/emoji/v2/svg/1f38a.svg
https://pbs.twimg.com/media/EzjeRUDXEAAf6VF?format=jpg&name=small
ICOM UAE
@UaeIcom
·
1h
Hi there, 
 TODAY 
You are invited to a Zoom meeting. 
When: Apr 22, 2021 09:30 PM Dubai 

Register in advance for this meeting:
https://us02web.zoom.us/meeting/register/tZMpdO2urDwsHdXy-qgCt5zN0nmT0b-hbqfp… 

After registering, you will receive a confirmation email containing information about joining the meeting.
1h
https://pbs.twimg.com/profile_images/1082518066884276226/ssAGhbqh_bigger.jpg
https://pbs.twimg.com/media/EzjcRu1UcAAdJDM?format=jpg&nam

Al Día
@aldiacsuf
·
3h
Register for this panel here:
https://fullerton.zoom.us/meeting/register/tZclc-2hrT8sG9WvaWMzeLzPXcFGzbiUAutC…
Quote Tweet
CSUF LCI
@CSUF_LCI
 · Apr 21
Amazing speakers lined up at this year’s COMM Week! 
 
Hear from Olivia Liendo, the Senior Manager of Digital News for Univision, next Tuesday on April 27 from 2:00-3:30 p.m. PT! 
Show this thread
1
2
3h
https://pbs.twimg.com/profile_images/1174358439524360192/KaI1f09M_bigger.jpg
Al Día
@aldiacsuf
·
3h
Replying to 
@CSUF_LCI
 @CSUF_CommWeek
 and 4 others
Register for this panel here:
Welcome! You are invited to join a meeting: Univision Panel: Olivia Liendo. After registering, you...
Welcome! You are invited to join a meeting: Univision Panel: Olivia Liendo. After registering, you will receive a confirmation email about joining the meeting.
fullerton.zoom.us
1
3h
https://pbs.twimg.com/profile_images/494989356092772353/aQ9vZgyf_bigger.jpeg
Trinity Univ of Asia
@TrinityUofA
·
3h
2/2 Register Now! It’s FREE!
https://

...

Language Center @ Harvard University
@LangCtrHarvard
·
17h
Language PhDs, please join us on 4/28 for a roundtable discussion about alternatives to academic careers with professionals from language technology. Register here: https://harvard.zoom.us/meeting/register/tJwsd-CgrDItE9J9t1Dnsf1nVonUxWuXcDyY…
1
17h
https://pbs.twimg.com/profile_images/908728254445621248/NsMAAECU_bigger.jpg
https://pbs.twimg.com/media/EzgPjjzXMAIp-Kk?format=jpg&name=small
Adelphi Library
@AdelphiLibrary
·
17h
Join us!! Sign up and register for this event!! https://adelphiuniversity.zoom.us/meeting/register/tJwvceuvqjMiGtZLsw3hRuXryys9krxkyW31… #adelphi #library #adelphievents #zoom
17h
https://pbs.twimg.com/profile_images/1296130721975808002/ULA3J7ET_bigger.jpg
https://pbs.twimg.com/media/EzgPjEiVoAIRF2z?format=jpg&name=small
10ksbDallasCollege
@10ksbDalCollege
·
17h
Join us at Financial Forecasting for Small Businesses: A Tool for Recovery and Growth tomorrow at 1:00 pm. Register at this link: http://bit.ly/3w

familywealthtoday
@familywealthtdy
·
18h
Join us Saturday April 24th @ 10AM to learn how you can start a career in Life Insurance. Register today: https://bit.ly/LifeInsuranceCareer…
18h
https://pbs.twimg.com/profile_images/1366215872/agaglogo150_bigger.jpg
https://pbs.twimg.com/media/EzgMkqLXEAIMAQu?format=jpg&name=240x240
AGAG
@agagafrica
·
18h
At 1030a EST we will begin our #conversation w/ Jesse Eaves and Dr. Akwasi Aidoo of 
@HumanityUnited
 re: Supporting #Networks in Support of #Change. Join us for live tweets using hashtag #AGAGNetworks

https://us02web.zoom.us/meeting/register/tZwpduurqT0jG91RnfcG94r-lJhN8D9n3ZXo…
Humanity United
1
18h
https://pbs.twimg.com/profile_images/1131543782158061568/dnT-XP8y_bigger.png
https://pbs.twimg.com/media/EzgMbJnVgAUIDIc?format=jpg&name=small
Vijaybhoomi University
@Vijaybhoomi_Uni
·
18h
Keeping in mind the growing need to integrate new-age #skills into the mainstream #school #curriculum, 
@Vijaybhoomi_Uni
 is organizing a #colloquim on #Liber

UN Peacebuilding
@UNPeacebuilding
·
18h
Community Engagement for Resilience & Peace!
Examining role of community engagement approaches in strengthening resilience & facilitating peacebuilding in conflict-affected & fragile contexts w/ 
@CoumbaDSow
 @FAOWestAfrica
 @SheinbergUN
 
22 April 
See here: https://bit.ly/3t2ptv9
8
12
18h
https://pbs.twimg.com/profile_images/2689511937/0337b6fe9f291ab5059eb531214d1d0c_bigger.png
https://pbs.twimg.com/media/EzgKE3fXsAIxqSS?format=jpg&name=small
SJSU ALASC
@ALASCSJSU
·
18h
Please join us for a special presentation with guest speaker Tina McPherson who will explore how librarians can contribute to community resilience work by creating a culture of respect, understanding, and empathy on 4/27 at 6pm PT. Register via Zoom: https://sjsu.zoom.us/meeting/register/tZcldOGtrTwqE9HRxGdbnMX5wxEfUA5Nl6-l…
1
18h
https://pbs.twimg.com/profile_images/1264857726754185216/ADC8RySm_bigger.jpg
https://abs-0.twimg.com/emoji/v2/svg/1f447.svg
PPMA_HR
@PPMA_HR
·
18h
On

Teacher's Discovery
@_TDSocial
·
18h
Join us tonight to learn how to effectively use novels and readers in your World Language classroom! The best part? It's FREE!

Wednesday, April 21 at 5:30 p.m. Eastern Time (Detroit). 
Register Now! #langchat #languageteacher

https://teachersdiscovery.zoom.us/meeting/register/tZMocu2gqjgtGtfUjd1KUiaGQQtipbxHrq4l…
18h
https://pbs.twimg.com/profile_images/1383660245910638592/ZxO568Jm_bigger.jpg
https://pbs.twimg.com/media/EzgFU3EXMAIPBRI?format=jpg&name=small
Saddam Sayyaleh
@SaddamSayyaleh
·
18h
It's not too late to register for the #MENA Youth Community of Practice's webinar tomorrow at 9 AM EST on #youth engagement during #COVID19 featuring the founder of 
@ILearnJo
. Register to attend here: 

https://us02web.zoom.us/meeting/register/tZAkcOCqrTMrHdKbmZGeaydDtVkJv54-O2F4…
1
18h
https://pbs.twimg.com/profile_images/1256051702404440066/2-TwoRf5_bigger.jpg
https://abs-0.twimg.com/emoji/v2/svg/1f449.svg
https://pbs.twimg.com/media/EzgDU3IXIAEf2Vf?for

WebDriverException: Message: chrome not reachable
  (Session info: chrome=89.0.4389.128)


In [4]:
import pandas as pd
df=pd.read_csv("C:\\Users\\Admin\\Twitter_apr22.csv")
df

,PostDate,Description,image_url,source_url
0,2021-04-22T06:53:04.000Z,Cheche\n@WinnieCheche\n·\n10s\nReplying to \n@...,NaN,https://twitter.com/WinnieCheche/status/138512...
1,2021-04-22T06:51:00.000Z,SITE_Africa\n@site_africa\n·\n2m\nIt's Earth D...,https://pbs.twimg.com/media/EzjyvydXEAATEhM?fo...,https://twitter.com/site_africa/status/1385123...
2,2021-04-22T06:51:00.000Z,Church of England Environment Programme\n@CofE...,NaN,https://twitter.com/CofEEnvironment/status/138...
3,2021-04-22T06:50:51.000Z,CefES Research Center\n@CefesCenter\n·\n2m\nOu...,https://pbs.twimg.com/media/EzjyyxuWYAE88Cs?fo...,https://twitter.com/CefesCenter/status/1385123...
4,2021-04-22T06:48:03.000Z,Monthly Review\n@monthly_review\n·\n5m\n#Trial...,https://pbs.twimg.com/media/EzjyOVqXoAITTK_?fo...,https://twitter.com/monthly_review/status/1385...
...,...,...,...,...
684,2021-04-21T13:42:16.000Z,Coach P\n@CoachP_Irish90\n·\n18h\nJoin us \n@N...,NaN,https://twitter.com/CoachP_Irish90/status/1384...
685,2021-04-21T13:34:27.000Z,AFP SouthEastOntario\n@AFP_SEO\n·\n18h\nJoin \...,https://pbs.twimg.com/media/EzgFoqlXoAAuIX-?fo...,https://twitter.com/AFP_SEO/status/13848631025...
686,2021-04-21T13:33:36.000Z,Teacher's Discovery\n@_TDSocial\n·\n18h\nJoin ...,https://pbs.twimg.com/media/EzgFSv0WUAAczvq?fo...,https://twitter.com/_TDSocial/status/138486288...
687,2021-04-21T13:33:02.000Z,Saddam Sayyaleh\n@SaddamSayyaleh\n·\n18h\nIt's...,https://pbs.twimg.com/media/EzgFU3EXMAIPBRI?fo...,https://twitter.com/SaddamSayyaleh/status/1384...


In [5]:
import re
zoom_regex= re.compile('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+')

In [6]:
Lst=[]

In [7]:
#x=zoom_regex.findall(tweet_text)
for i in df['Description']:
    Lst.append(zoom_regex.findall(str(i)))

In [8]:
df['Zoom_link']=Lst

In [9]:
df.to_csv("Twitter_apr22_zoom.csv", index=False)
df

,PostDate,Description,image_url,source_url,Zoom_link
0,2021-04-22T06:53:04.000Z,Cheche\n@WinnieCheche\n·\n10s\nReplying to \n@...,NaN,https://twitter.com/WinnieCheche/status/138512...,[]
1,2021-04-22T06:51:00.000Z,SITE_Africa\n@site_africa\n·\n2m\nIt's Earth D...,https://pbs.twimg.com/media/EzjyvydXEAATEhM?fo...,https://twitter.com/site_africa/status/1385123...,[https://us02web.zoom.us/meeting/register/tZYl...
2,2021-04-22T06:51:00.000Z,Church of England Environment Programme\n@CofE...,NaN,https://twitter.com/CofEEnvironment/status/138...,[https://zoom.us/meeting/register/tJYvc-2prT4t...
3,2021-04-22T06:50:51.000Z,CefES Research Center\n@CefesCenter\n·\n2m\nOu...,https://pbs.twimg.com/media/EzjyyxuWYAE88Cs?fo...,https://twitter.com/CefesCenter/status/1385123...,"[https://tinyurl.com/r7a4vc3v, https://tinyurl..."
4,2021-04-22T06:48:03.000Z,Monthly Review\n@monthly_review\n·\n5m\n#Trial...,https://pbs.twimg.com/media/EzjyOVqXoAITTK_?fo...,https://twitter.com/monthly_review/status/1385...,[https://bit.ly/2RPmpVq]
...,...,...,...,...,...
684,2021-04-21T13:42:16.000Z,Coach P\n@CoachP_Irish90\n·\n18h\nJoin us \n@N...,NaN,https://twitter.com/CoachP_Irish90/status/1384...,[]
685,2021-04-21T13:34:27.000Z,AFP SouthEastOntario\n@AFP_SEO\n·\n18h\nJoin \...,https://pbs.twimg.com/media/EzgFoqlXoAAuIX-?fo...,https://twitter.com/AFP_SEO/status/13848631025...,[https://us02web.zoom.us/meeting/register/tZws...
686,2021-04-21T13:33:36.000Z,Teacher's Discovery\n@_TDSocial\n·\n18h\nJoin ...,https://pbs.twimg.com/media/EzgFSv0WUAAczvq?fo...,https://twitter.com/_TDSocial/status/138486288...,[https://teachersdiscovery.zoom.us/meeting/reg...
687,2021-04-21T13:33:02.000Z,Saddam Sayyaleh\n@SaddamSayyaleh\n·\n18h\nIt's...,https://pbs.twimg.com/media/EzgFU3EXMAIPBRI?fo...,https://twitter.com/SaddamSayyaleh/status/1384...,[https://us02web.zoom.us/meeting/register/tZAk...


In [10]:
import pandas as pd
df=pd.read_csv("Twitter_apr22_zoom.csv")
df

,PostDate,Description,image_url,source_url,Zoom_link
0,2021-04-22T06:53:04.000Z,Cheche\n@WinnieCheche\n·\n10s\nReplying to \n@...,NaN,https://twitter.com/WinnieCheche/status/138512...,[]
1,2021-04-22T06:51:00.000Z,SITE_Africa\n@site_africa\n·\n2m\nIt's Earth D...,https://pbs.twimg.com/media/EzjyvydXEAATEhM?fo...,https://twitter.com/site_africa/status/1385123...,['https://us02web.zoom.us/meeting/register/tZY...
2,2021-04-22T06:51:00.000Z,Church of England Environment Programme\n@CofE...,NaN,https://twitter.com/CofEEnvironment/status/138...,['https://zoom.us/meeting/register/tJYvc-2prT4...
3,2021-04-22T06:50:51.000Z,CefES Research Center\n@CefesCenter\n·\n2m\nOu...,https://pbs.twimg.com/media/EzjyyxuWYAE88Cs?fo...,https://twitter.com/CefesCenter/status/1385123...,"['https://tinyurl.com/r7a4vc3v', 'https://tiny..."
4,2021-04-22T06:48:03.000Z,Monthly Review\n@monthly_review\n·\n5m\n#Trial...,https://pbs.twimg.com/media/EzjyOVqXoAITTK_?fo...,https://twitter.com/monthly_review/status/1385...,['https://bit.ly/2RPmpVq']
...,...,...,...,...,...
684,2021-04-21T13:42:16.000Z,Coach P\n@CoachP_Irish90\n·\n18h\nJoin us \n@N...,NaN,https://twitter.com/CoachP_Irish90/status/1384...,[]
685,2021-04-21T13:34:27.000Z,AFP SouthEastOntario\n@AFP_SEO\n·\n18h\nJoin \...,https://pbs.twimg.com/media/EzgFoqlXoAAuIX-?fo...,https://twitter.com/AFP_SEO/status/13848631025...,['https://us02web.zoom.us/meeting/register/tZw...
686,2021-04-21T13:33:36.000Z,Teacher's Discovery\n@_TDSocial\n·\n18h\nJoin ...,https://pbs.twimg.com/media/EzgFSv0WUAAczvq?fo...,https://twitter.com/_TDSocial/status/138486288...,['https://teachersdiscovery.zoom.us/meeting/re...
687,2021-04-21T13:33:02.000Z,Saddam Sayyaleh\n@SaddamSayyaleh\n·\n18h\nIt's...,https://pbs.twimg.com/media/EzgFU3EXMAIPBRI?fo...,https://twitter.com/SaddamSayyaleh/status/1384...,['https://us02web.zoom.us/meeting/register/tZA...


In [11]:
#do not forget 
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from webdriver_manager.chrome import ChromeDriverManager
import webdrivermanager
driver = webdriver.Chrome(ChromeDriverManager().install())

[WDM] - ====== WebDriver manager ======


[WDM] - Current google-chrome version is 89.0.4389
[WDM] - Get LATEST driver version for 89.0.4389
[WDM] - Driver [C:\Users\Admin\.wdm\drivers\chromedriver\win32\89.0.4389.23\chromedriver.exe] found in cache


In [12]:
## multiple zoom links
#1
count=0
for i,j in enumerate(df['Zoom_link']):
    count+=1
    #if count==12:
     #   break
    try:
        print(count)
        m=j.replace(",","")
        n=m.replace("[","")
        n=n.replace("]","")
        n=n.replace("'","")
        print(m)
        l=n.split(" ")
        print(len(l))
        if len(l)>1:
            for k in range(len(l)):
                zoom=l[k]
                print(zoom)
                try:
                    driver.get(zoom)
                    print("hi")
                
                    topic=driver.find_elements_by_class_name('controls')
                    title=topic[1].text
                    print(title)
                    if title=="":
                        print("title is not there")
                        continue
                    else:
                        print(l[k])
                        df.loc[i,"Zoom_link"]=l[k]
                except:
                    print("Is this working?")
                    df.loc[i,"Zoom_link"]=""
                    continue
    except:
            pass

1
[]
1
2
['https://us02web.zoom.us/meeting/register/tZYlfu6urTIuEtdPrO4AY3Em8xP2wGjddekF']
1
3
['https://zoom.us/meeting/register/tJYvc-2prT4tGdb9a-sHHB_HesJwsOh1HDzF']
1
4
['https://tinyurl.com/r7a4vc3v' 'https://tinyurl.com/4ebzdmz9']
2
https://tinyurl.com/r7a4vc3v
hi
CefES Seminar - Marco Casari (May 3, 2021)
https://tinyurl.com/r7a4vc3v
https://tinyurl.com/4ebzdmz9
hi
Is this working?
5
['https://bit.ly/2RPmpVq']
1
6
['https://zoom.us/meeting/register/tJIqc-CurDgsE9AGlcfa5PifE25SHJrVpgI8']
1
7
['http://bit.ly/kfsdsoftskillstraining']
1
8
['https://bit.ly/3dJWRjG.' 'https://facebook.com/3718693561590968/videos/3718697208257270' 'https://bit.ly/3dJWRjG']
3
https://bit.ly/3dJWRjG.
hi
Stakeholders Strategic Conversation on Migrant Workers' Franchise Rights
https://bit.ly/3dJWRjG.
https://facebook.com/3718693561590968/videos/3718697208257270
hi
Is this working?
https://bit.ly/3dJWRjG
hi
Stakeholders Strategic Conversation on Migrant Workers' Franchise Rights
https://bit.ly/3dJWRjG
9
['h

hi
Is this working?
http://ow.ly/fWda50EsClN.
hi
Lets Talk! Sex, HIV, STIs? Workshop
http://ow.ly/fWda50EsClN.
101
['https://zoom.us/meeting/register/tJ0udO6pqT8rGtSYGYFifUIOHBliJDdChWPX']
1
102
['https://kansas.zoom.us/meeting/register/tJUtfuqqrDIrHdcyRZff395P1TsM4uqj-eMF']
1
103
['https://us02web.zoom.us/meeting/register/tZUlfuqvqDwrGdKKDDePOcsY4fK8lvnXDhZQ']
1
104
['http://bit.ly/DASANmeet']
1
105
['https://fb.me/e/2g91UZ7FE' 'http://bit.ly/FS-cwaa-fcp21']
2
https://fb.me/e/2g91UZ7FE
hi
Is this working?
http://bit.ly/FS-cwaa-fcp21
hi
Journal launch: Flies, Cockroaches and Poets
http://bit.ly/FS-cwaa-fcp21
106
['https://bit.ly/3gp8Lm1']
1
107
['https://us06web.zoom.us/meeting/register/tJIrcu2tpzkoGddMP7FvsRofBwNAv8AZtyJR']
1
108
['http://bit.ly/REACHingUSFamilyAffair']
1
109
['http://tinyurl.com/SOS-StopAntiAsianHate']
1
110
['http://bit.ly/clo-yvm']
1
111
['http://bit.ly/GUPhillyC2C2021']
1
112
['https://tinyurl.com/hzeccmky']
1
113
[]
1
114
['https://us02web.zoom.us/meeting/registe

hi
Is this working?
https://csueb.zoom.us/meeting/register/tZ0qdeiopj8tGNSUviuEiEKD3teVq6MhDoJk
hi
STORK Journal Club:Generalizability Crisis
https://csueb.zoom.us/meeting/register/tZ0qdeiopj8tGNSUviuEiEKD3teVq6MhDoJk
228
[]
1
229
[]
1
230
['https://bit.ly/2PamwtD.']
1
231
[]
1
232
['https://zoom.us/meeting/register/tJEqf-6uqz0sGdNJxLuRgeO4gZx8yM2_NAiL?timezone_id=America%2FNew_York']
1
233
['https://zoom.us/meeting/register/tJElf-6przIsGN0Du1JasBoCCpJGzh8TA3Yj']
1
234
[]
1
235
['http://ow.ly/s3r350EuumR']
1
236
['http://bit.ly/frontlinesclimate']
1
237
['http://tiny.cc/lunchnlearn_sustain']
1
238
['https://bit.ly/2QNs3Xm']
1
239
[]
1
240
['https://us02web.zoom.us/meeting/register/tZYuduuurTIiHNXzE2NMf9gKLpW0aPFrlh3J']
1
241
['http://ow.ly/Rrut50EuuIH']
1
242
[]
1
243
['http://t.ly/kR7f']
1
244
['https://zoom.us/meeting/register/tJUkceurqD8tEt0imO9blCZxydiCTDE2UgBD']
1
245
[]
1
246
['http://bit.ly/VVapr29briefing']
1
247
['https://bit.ly/3gGtSAH']
1
248
['https://bit.ly/3ajeupA']
1
249

hi
Storytelling as Organizing Skill Share
http://bit.ly/bbbtestimony.
http://bit.ly/bbbtestimony
hi
Storytelling as Organizing Skill Share
http://bit.ly/bbbtestimony
330
['https://bit.ly/2RPmpVq']
1
331
['https://bit.ly/2Ql261G']
1
332
['https://tinyurl.com/4cjdjb6b']
1
333
['http://tinyurl.com/letsplantnj']
1
334
['http://bit.ly/CIWiLrs']
1
335
['https://hubs.ly/H0LKqd90']
1
336
['https://us02web.zoom.us/meeting/register/tZckcOuqrjotGtQEb_As2dOFVC69R888DNDv' 'https://urban.org/policy-centers/center-education-data-and-policy/projects/call-proposals-data-driven-analysis-timely-education-policy-topics']
2
https://us02web.zoom.us/meeting/register/tZckcOuqrjotGtQEb_As2dOFVC69R888DNDv
hi
Call for Proposals: Data-Driven Analysis of Timely Education Policy Topics
https://us02web.zoom.us/meeting/register/tZckcOuqrjotGtQEb_As2dOFVC69R888DNDv
https://urban.org/policy-centers/center-education-data-and-policy/projects/call-proposals-data-driven-analysis-timely-education-policy-topics
hi
Is this wo

hi
Obesity Management ECHO Session #4.4
http://echo.zoom.us/meeting/register/tJwtce2uqDIoHtCxLsMd1MrCfXA5vtH-P3NK
http://echo.zoom.us/meeting/register/tJwtce2uqDIoHtCxLsMd1MrCfXA5vtH-P3NK
hi
Obesity Management ECHO Session #4.4
http://echo.zoom.us/meeting/register/tJwtce2uqDIoHtCxLsMd1MrCfXA5vtH-P3NK
456
['http://ow.ly/WPi050Eu5li']
1
457
[]
1
458
['https://bit.ly/3kKQiQ5' 'https://bit.ly/3kKQiQ5']
2
https://bit.ly/3kKQiQ5
hi
Join us for Friday's Power Lunch!

AGENDA WILL BE ANNOUNCED

Don't miss our informative, current, upbeat hour with legislators, candidates, issues, music and door prizes!
https://bit.ly/3kKQiQ5
https://bit.ly/3kKQiQ5
hi
Join us for Friday's Power Lunch!

AGENDA WILL BE ANNOUNCED

Don't miss our informative, current, upbeat hour with legislators, candidates, issues, music and door prizes!
https://bit.ly/3kKQiQ5
459
['https://us02web.zoom.us/meeting/register/tZ0pc-6hqzIrH9SP1Pflv5e-VIvbk-VsdbvS']
1
460
['http://bit.ly/untlib-mindfulstudybreaks']
1
461
['http://bit.l

hi
Is this working?
571
['https://uni.zoom.us/meeting/register/tJMlfu2gqDwtGtD10A5BiQdqbtLZmB9ZMhRz']
1
572
['https://fal.cn/3eRYA']
1
573
['http://ibr.news/multimodal']
1
574
['https://utah.zoom.us/meeting/register/tJYrf-2tpzMjGdby6SHOisOucI9wTGwjV3p7']
1
575
['http://bit.ly/acalaunch']
1
576
['https://bit.ly/3de0T51']
1
577
['http://ow.ly/UsVX50EtQhr']
1
578
['http://ow.ly/PyPd50Etaqz']
1
579
['https://fal.cn/3eRTs' 'https://fal.cn/3eRTt']
2
https://fal.cn/3eRTs
hi
Is this working?
https://fal.cn/3eRTt
hi
As your time as a student at Montclair State University comes to an end, let us be the first to celebrate you and welcome you to your alumni network!

Join the Office of Alumni Engagement and Annual Giving for a celebration of the Class of 2021! The event will include trivia, prizes and a special opportunity to celebrate your accomplishments as a Red Hawk!
https://fal.cn/3eRTt
580
['https://us02web.zoom.us/meeting/register/tZckc-utrTkiE9zHPcOqlFF5eHNEqyIEazEF']
1
581
['http://Twitch

hi
Recovery Corps Info Session
https://us02web.zoom.us/j/87879290106
https://sites.google.com/qc.cuny.edu/servicecorps/upcoming-opportunities/cuny-recovery-corps
hi
Is this working?
678
[]
1
679
['http://bit.ly/3wCLJh5']
1
680
[]
1
681
['http://tinyurl.com/UKpickering' 'http://uky.edu/chellgren/pickeringrangel-fellowship-information-session?j=343704&sfmc_sub=127479599&l=20904_HTML&u=10422805&mid=10966798&jb=0']
2
http://tinyurl.com/UKpickering
hi
Pickering/Rangel Fellowship Information Session
http://tinyurl.com/UKpickering
http://uky.edu/chellgren/pickeringrangel-fellowship-information-session?j=343704&sfmc_sub=127479599&l=20904_HTML&u=10422805&mid=10966798&jb=0
hi
Is this working?
682
['https://us02web.zoom.us/meeting/register/tZ0qf-iqpj0rE9EwqopczXHJfx3qRFSHk-SV']
1
683
['http://bit.ly/CaMSIGN-WiN' 'http://bit.ly/CaMSIGN-WiN']
2
http://bit.ly/CaMSIGN-WiN
hi
Reflections, Resilience and Path to Success: A Panel with Women Healthcare Leaders
http://bit.ly/CaMSIGN-WiN
http://bit.ly/CaMS

In [13]:
#entering title and start time
#2
count=0
for i,j in enumerate(df['Zoom_link']):
    count+=1
    #if count==12:
        #break
    try:
        print(count)
        m=j.replace(",","")
        n=m.replace("[","")
        n=n.replace("]","")
        n=n.replace("'","")
        print(m)
        l=n.split(" ")
        if len(l)==1:
            driver.get(l[0])
            topic=driver.find_elements_by_class_name('controls')
            title=topic[0].text
            l=title.split("\n")
            if len(l)>2:
                x=topic[1].text
                n=x.split("\n")
                if len(n)==1:
                    title=x
            print(title)
            time=topic[3].text
            if time=="":
                time=topic[2].text

            print(i,time)
            df.loc[i,"start_time"]=time
            df.loc[i,"event_title"]=title
    except:
        pass

1
[]
2
['https://us02web.zoom.us/meeting/register/tZYlfu6urTIuEtdPrO4AY3Em8xP2wGjddekF']
EARTH DAY SHOWCASE AFRICA
1 Apr 22, 2021 11:00 AM in Johannesburg
3
['https://zoom.us/meeting/register/tJYvc-2prT4tGdb9a-sHHB_HesJwsOh1HDzF']
Webinar on "To replace or not replace?" your church heating and lighting, from the Church of England Environment Programme
2 Apr 27, 2021 04:00 PM in London
4

5
['https://bit.ly/2RPmpVq']
Virtual Book Launch: Michael Tigar's "Sensing Injustice"
4 Apr 23, 2021 12:00 PM in Central Time (US and Canada)
6
['https://zoom.us/meeting/register/tJIqc-CurDgsE9AGlcfa5PifE25SHJrVpgI8']
POPIA Public Consultation Forum
5 May 3, 2021 10:00 AM in Johannesburg
7
['http://bit.ly/kfsdsoftskillstraining']
SOFT SKILL TRAINING FOR STUDENTS OF TERTIARY INSTITUTIONS
6 Time shows in West Central Africa
8
https://bit.ly/3dJWRjG
Stakeholders Strategic Conversation on Migrant Workers' Franchise Rights
7 Apr 22, 2021 02:00 PM in Taipei
9
['http://bit.ly/3w4CxBY']
INBIOSIS Webinar Lectur

VMSS + OCM Meet, Eat, & Greet!
73 Apr 28, 2021 12:00 PM in Mountain Time (US and Canada)
75
[]
76
['https://tinyurl.com/9thStTraffic']
9th St. Traffic Calming Neighborhood Workshop
75 Apr 22, 2021 05:30 PM in Pacific Time (US and Canada)
77
['https://us02web.zoom.us/meeting/register/tZAvd--pqDIqHtVyTe0Cw101RIbpMg-kCZgo']
The JFK Assassination in History
76 Apr 21, 2021 06:00 PM in Arizona
78

79
['https://us02web.zoom.us/meeting/register/tZAufuCqrz4tEtHnvV4ZT0iKDnZyTUaFgKiI']
Wise Wednesday
78 Apr 21, 2021 06:00 PM in Arizona
80
['https://bit.ly/3ekb07E']
Earth Day Energy Resources
79 Apr 22, 2021 11:00 AM in Eastern Time (US and Canada)
81
['http://bit.ly/SPARPanel']
SPHAC The Future of Public Health Fireside Chat #2
80 Apr 22, 2021 07:00 PM in Mountain Time (US and Canada)
82
['https://zoom.us/meeting/register/tJMlf-qurT0sHdS2Bujvcm6hsvjuKYuwsIJq']
#ServeHope: East Coast WebIftar
81 Apr 21, 2021 08:30 PM in Eastern Time (US and Canada)
83
['https://buff.ly/3uUA9MO']
Making Health and

100% panel
134 Apr 22, 2021 06:00 PM in Eastern Time (US and Canada)
136
['http://ow.ly/Mnee50EulIq']
LCDP VPB
135 Time shows in Central Time (US and Canada)
137
['http://bit.ly/REDSHOEzumba2']
CAC -R.E.D. Shoe Fitness
136 Apr 24, 2021 09:00 AM in Eastern Time (US and Canada)
138
['https://bit.ly/2QuW66h']
Advocacy 101
137 Apr 21, 2021 06:00 PM in Pacific Time (US and Canada)
139
['https://bit.ly/3tFGCuF']
All-In Heisman Club Happy Hour
138 Apr 21, 2021 07:30 PM in Eastern Time (US and Canada)
140
['http://ow.ly/vJh650EoOFC']
Crack It Open: Engaging Pandemic, Racism, and the Church Cracked Open
139 Apr 21, 2021 06:30 PM in Eastern Time (US and Canada)
141
['https://bit.ly/foraunion']
General Body Meeting on April 21 / Reunión de cuerpo general el abril 21
140 Apr 21, 2021 08:00 PM in Eastern Time (US and Canada)
142
['https://us02web.zoom.us/meeting/register/tZwvcOGsqDgtHNXKBTFrORmjjz5qyw8s2uRg']
Poverty Talks! Community Conversation!
141 May 12, 2021 06:00 PM in Mountain Time (US and 

European Seminar: Alex Lichtenstein
203 May 5, 2021 02:00 PM in Eastern Time (US and Canada)
205
['http://bit.ly/westchester-open-meeting']
Westchester/Hudson Valley Membership Meeting
204 Time shows in Eastern Time (US and Canada)
206
['https://bit.ly/3ezPqfF.']
The Impact of Biden's China Policy on the Pacific Northwest
205 Apr 22, 2021 04:00 PM in Pacific Time (US and Canada)
207
[]
208
['http://bit.ly/3eiES4o']
Clemson COVID Challenge
207 Apr 23, 2021 12:00 PM in Eastern Time (US and Canada)
209
['http://ow.ly/5C5Y50DyoP7']
FBC Evening Bible Study Wednesday @ 6:30 PM
208 Time shows in Central Time (US and Canada)
210
['http://bit.ly/r2r133']
R2R Town Hall: 33rd & 1st Wards
209 Apr 22, 2021 06:00 PM in Central Time (US and Canada)
211
['https://rfr.bz/t2c4jjv']
Delivering Powerful, Persuasive Presentations.
210 Apr 28, 2021 02:00 PM in Eastern Time (US and Canada)
212
['https://us02web.zoom.us/meeting/register/tZMlfuCsqzMoGNZIE1gxdAp6vx7HChESGfoJ']
Black Lawrence Press Virtual Readi

Animal Health Emergency Management webinar
268 May 17, 2021 12:00 PM in Mountain Time (US and Canada)
270
['https://nyu.zoom.us/meeting/register/tJYvdO2upjgiEtaSfTVb4fWyO1ozy-KRGlTl']
Art Book Series - Olav Velthuis with Diana Seave Greenwald, Painting by Numbers
269 Apr 23, 2021 09:00 AM in Eastern Time (US and Canada)
271
['https://bit.ly/3vb057c']
Engaging Youth to Prevent Violence Against Children / Mobiliser les jeunes pour prévenir la violence faite aux enfants
270 May 6, 2021 12:00 PM in Eastern Time (US and Canada)
272
['https://buff.ly/2QkMsDw']
LET'S GET TOGETHER!!
271 Apr 21, 2021 07:00 PM in Edmonton
273
['https://bit.ly/32CUie5']
NAMI with PGRC
272 Apr 22, 2021 06:00 PM in Eastern Time (US and Canada)
274
['https://zoom.us/meeting/register/tJYodeCrpjojHdIpsq7Yu91G5voR765HOzB2']
Epilepsy Community Resource Series Session 3: Epilepsy, Employment, and Employee Rights
273 May 1, 2021 09:30 AM in Eastern Time (US and Canada)
275
['http://ow.ly/aLKt50Etn43']
Promoting Your Artwo

...

Prepare for Google's Core Web Values or Face their Wrath!
396 Apr 22, 2021 06:00 PM in Eastern Time (US and Canada)
398
['https://bit.ly/3dI84CR']
Rebuild City College presents a Faculty Empowerment Fair
397 Apr 24, 2021 01:00 PM in Pacific Time (US and Canada)
399
['https://nyu.zoom.us/meeting/register/tJwqc-GrpjkoHtwxBZqs3JN_ClGtopRc_CH4']
Post-Chauvin Verdict Staff and Faculty Reflection
398 Apr 23, 2021 03:00 PM in Eastern Time (US and Canada)
400

401
['https://us02web.zoom.us/meeting/register/tZctce-rpzMrHdcR-y5JZKPiDNy09i2nmnMI']
Paid Leave and Special Needs
400 Apr 21, 2021 06:00 PM in Eastern Time (US and Canada)
402
[]
403
[]
404
[]
405
[]
406
['http://bit.ly/ThaYard']
Tha Yard: Weekly Hangout
405 Apr 22, 2021 05:00 PM
Apr 29, 2021 05:00 PM
May 6, 2021 05:00 PM
407
['https://miami.zoom.us/meeting/register/tJMofu2rrzkpHdGRP43B6r3dSyZYQFajKHw7']
“Acceleration? Digital Resources and the Speed of Scholarship” by Dr. Jeffrey Ravel
406 Apr 23, 2021 10:30 AM in Eastern Time (US and 

Adam Smith Workshop Spring 2021
469 Apr 22, 2021 02:00 PM
Apr 23, 2021 02:00 PM
471
['http://ow.ly/1foB50Egf47']
SVN Lunch n Learn April 29th 2021
470 Apr 29, 2021 12:00 PM in Arizona
472
['http://bit.ly/TUPoliceBill']
Why Trade Unions should Oppose the Police Bill
471 Apr 22, 2021 05:30 PM in London
473
['http://bit.ly/3dmTLDq']
IMF World Economic Outlook
472 Apr 23, 2021 01:30 PM in Eastern Time (US and Canada)
474
['http://bit.ly/ChildCareOnABudget']
Navigating Quality Child Care on a Budget
473 Apr 27, 2021 06:30 PM in Eastern Time (US and Canada)
475
['http://tinyurl.com/yf94e5rx']
My Meeting
474 Apr 24, 2021 11:00 AM in Arizona
476
['https://bit.ly/3aaYXbB']
CLM Peer Advisory Group - Customer Success
475 Apr 21, 2021 02:00 PM in Central Time (US and Canada)
477
['https://uwyo.zoom.us/meeting/register/tJYudOygrz0oHNWsQ9sqZ-LCB4_E2N4oV-TJ']
Pole Mountain Gateways Public Meeting
476 Apr 22, 2021 05:30 PM in Mountain Time (US and Canada)
478
['http://tinyurl.com/mccaskeydual.']
479
[

Supply Chain Management Graduate Program Information Session
527 Apr 21, 2021 04:00 PM in Eastern Time (US and Canada)
529
['https://go.illinois.edu/FoodSecurity2021']
2021 Illinois Global Food Security Summit
528 Apr 29, 2021 09:00 AM in Central Time (US and Canada)
530
[]
531
[]
532
[]
533
['https://upenn.zoom.us/meeting/register/tJAof-uuqjwjE9e5bc7bK3ajXLubM_GPal3n']
Transversal Presents Transcreations: Translation and Creative Writing
532 Apr 28, 2021 06:00 PM in Eastern Time (US and Canada)
534
['https://ravenswoodschools.zoom.us/meeting/register/tJYsde-orT0uEtJuooN3owclIfirHk5YuitX']
Magical Bridge @ the Cesar Chavez Ravenswood Middle School
533 Apr 21, 2021 06:00 PM in Pacific Time (US and Canada)
535
[]
536
['https://atheists.org/biasedcourts']
The Fight Ahead: Unstacking the Ideologically Biased Courts
535 Apr 21, 2021 08:00 PM in Eastern Time (US and Canada)
537
['https://bit.ly/afro-latin-american-studies']
Conversatorio: Afro-Latin American Studies and its Institutionalizat

Spotlight on Care
586 Time shows in Central Time (US and Canada)
588
['http://bincom.net/emigr8openday']
eMigr8 Open Day - May 5 2021
587 May 5, 2021 08:00 PM in West Central Africa
589
https://zoom.us/meeting/register/tJEkduyvrDIrHNOQ_RjOxCDFa6UceEyFlFS3
Orlando VMUG - April 2021
588 Apr 22, 2021 11:00 AM in Eastern Time (US and Canada)
590
https://tinyurl.com/WMW2021SpeakOut
Workers' Memorial Week National Speak Out / Manifestación nacional de la Semana Conmemorativa de lxs Trabajadorxs
589 Apr 27, 2021 02:00 PM in Eastern Time (US and Canada)
591
['https://tinyurl.com/RRCMicase']
RRC + Mi Casa
590 Apr 28, 2021 06:00 PM in Eastern Time (US and Canada)
592
['http://myumi.ch/0WmBO']
City of Champions: An Exploration of Detroit as a Sports Town
591 Apr 22, 2021 06:00 PM in Eastern Time (US and Canada)
593
['http://ow.ly/euha50Eua5W']
Dr. Chris Wilson Trauma Informed Training
592 Apr 21, 2021 01:00 PM in Eastern Time (US and Canada)
594
[]
595
https://dsausa.zoom.us/meeting/register/tJ0k

ISL Circles with Career Center and Office of Global Engagement: "International Students' Career Opportunities in the U.S."
648 Apr 28, 2021 03:30 PM in Eastern Time (US and Canada)
650
['https://zoom.us/meeting/register/tJwkdempqjgvE9eOFkbGY5dQSslYnWhL6DOU']
RiSWP Mentorship Series | EP5: Riccardo Taormina
649 Apr 21, 2021 05:00 PM in Jerusalem
651
https://bit.ly/3clPboW.
Poetic Journeys Release Party
650 Apr 21, 2021 07:00 PM in Eastern Time (US and Canada)
652
['https://bit.ly/3rhGjV7']
COVID-19- Vaccines, Remote Working and Leave Issue, The Latest Developments Webinar
651 Apr 22, 2021 10:00 AM in Eastern Time (US and Canada)
653
['http://ow.ly/ZE2I50EtGZC']
THCC Town Hall Meeting
652 Apr 29, 2021 10:00 AM
Jun 24, 2021 10:00 AM
Aug 26, 2021 10:00 AM
Oct 28, 2021 10:00 AM
Dec 9, 2021 10:00 AM
Feb 24, 2022 10:00 AM
Apr 28, 2022 10:00 AM
Jun 30, 2022 10:00 AM
654
['https://bit.ly/LifeInsuranceCareer']
Family Wealth Group Insurance Wealth Agent Opportunity
653 Please choose only one meet

In [14]:
#multiple start time code
#3
#3
count=0
for i,j in enumerate(df["start_time"]):
    count+=1
    #if count==11:
    #    break
    print(type(j))
    try:
        t=j.split("\n")
        n=len(t)
        print(t[n-1])
        df.loc[i,"start_time"]=t[n-1]
    except:
        pass

<class 'float'>
<class 'str'>
Apr 22, 2021 11:00 AM in Johannesburg
<class 'str'>
Apr 27, 2021 04:00 PM in London
<class 'float'>
<class 'str'>
Apr 23, 2021 12:00 PM in Central Time (US and Canada)
<class 'str'>
May 3, 2021 10:00 AM in Johannesburg
<class 'str'>
Time shows in West Central Africa
<class 'str'>
Apr 22, 2021 02:00 PM in Taipei
<class 'str'>
Apr 23, 2021 03:00 PM in Kuala Lumpur
<class 'str'>
May 15, 2021 01:30 PM in London
<class 'float'>
<class 'str'>
Apr 25, 2021 06:00 PM in West Central Africa
<class 'str'>
Apr 22, 2021 01:00 PM in Eastern Time (US and Canada)
<class 'float'>
<class 'str'>
Time shows in Canberra, Melbourne, Sydney
<class 'str'>
Apr 22, 2021 02:00 PM in London
<class 'str'>
Apr 22, 2021 09:30 PM in Dubai
<class 'str'>
Apr 22, 2021 07:00 PM in Mountain Time (US and Canada)
<class 'float'>
<class 'str'>
Apr 22, 2021 07:30 PM in Nairobi
<class 'float'>
<class 'str'>
Apr 22, 2021 10:00 AM in Nairobi
<class 'float'>
<class 'str'>
Apr 23, 2021 07:00 PM in Ind

<class 'float'>
<class 'str'>
Apr 22, 2021 11:00 AM in Pacific Time (US and Canada)
<class 'str'>
Apr 22, 2021 11:45 AM in Pacific Time (US and Canada)
<class 'str'>
Apr 22, 2021 01:00 PM in Eastern Time (US and Canada)
<class 'float'>
<class 'str'>

<class 'str'>
Apr 22, 2021 03:00 PM in Mountain Time (US and Canada)
<class 'str'>
Apr 22, 2021 05:00 PM in Eastern Time (US and Canada)
<class 'str'>

<class 'str'>
Apr 22, 2021 05:30 PM in Eastern Time (US and Canada)
<class 'str'>
Apr 22, 2021 05:00 PM in Pacific Time (US and Canada)
<class 'str'>
Apr 21, 2021 05:30 PM in Central Time (US and Canada)
<class 'str'>
Apr 22, 2021 12:00 PM in Eastern Time (US and Canada)
<class 'float'>
<class 'str'>
May 6, 2021 11:00 AM in Mountain Time (US and Canada)
<class 'str'>

<class 'str'>
Apr 22, 2021 05:30 PM in Eastern Time (US and Canada)
<class 'str'>
Apr 24, 2021 11:00 AM in Eastern Time (US and Canada)
<class 'str'>
Time shows in Mountain Time (US and Canada)
<class 'str'>
Apr 30, 2021 12:30

<class 'float'>
<class 'str'>
Apr 28, 2021 04:00 PM in Mexico City
<class 'str'>
Apr 21, 2021 07:00 PM in Eastern Time (US and Canada)
<class 'str'>
Apr 26, 2021 05:00 PM in Pacific Time (US and Canada)
<class 'str'>
Apr 21, 2021 05:00 PM in Eastern Time (US and Canada)
<class 'float'>
<class 'str'>
Apr 22, 2021 11:00 AM in Eastern Time (US and Canada)
<class 'str'>
Apr 24, 2021 05:00 PM
<class 'str'>
May 28, 2021 12:00 PM
<class 'float'>
<class 'str'>
Apr 23, 2021 08:30 AM
<class 'str'>
Apr 21, 2021 05:30 PM in Eastern Time (US and Canada)
<class 'str'>
Apr 22, 2021 01:00 PM in Eastern Time (US and Canada)
<class 'str'>
May 19, 2021 02:00 PM in London
<class 'str'>
Apr 28, 2021 06:30 PM in Eastern Time (US and Canada)
<class 'str'>
Apr 27, 2021 06:00 PM in Eastern Time (US and Canada)
<class 'str'>
Apr 29, 2021 01:00 PM in Eastern Time (US and Canada)
<class 'float'>
<class 'str'>
May 4, 2021 07:30 PM in Eastern Time (US and Canada)
<class 'str'>
Apr 22, 2021 01:00 PM in Eastern Time 

<class 'str'>
Apr 22, 2021 06:00 PM in Eastern Time (US and Canada)
<class 'str'>
Apr 21, 2021 05:00 PM in Central Time (US and Canada)
<class 'float'>
<class 'str'>
Apr 22, 2021 03:30 PM in Eastern Time (US and Canada)
<class 'str'>
May 5, 2021 12:00 PM in Eastern Time (US and Canada)
<class 'str'>
Apr 28, 2021 04:00 PM in Mexico City
<class 'str'>
Stay signed in
<class 'str'>
Apr 28, 2021 08:00 PM in Dublin
<class 'str'>
Apr 22, 2021 05:00 PM in Mexico City
<class 'str'>
May 5, 2021 07:00 PM in London
<class 'str'>
Apr 21, 2021 12:30 PM in Central Time (US and Canada)
<class 'str'>
Apr 23, 2021 09:00 AM in Eastern Time (US and Canada)
<class 'str'>
Apr 28, 2021 07:00 PM in Eastern Time (US and Canada)
<class 'str'>
Apr 22, 2021 06:00 PM in Eastern Time (US and Canada)
<class 'str'>
Apr 22, 2021 06:30 PM in Stockholm
<class 'float'>
<class 'float'>
<class 'str'>
Time shows in Eastern Time (US and Canada)
<class 'float'>
<class 'float'>
<class 'str'>
Apr 21, 2021 07:00 PM in Central Ti

In [15]:
#Remove list chize from zoomlinks
#4
c=0
for i,j in enumerate(df['Zoom_link']):
#     c+=1
#     if c==10:
#         break
    m=j.replace(",","")
    n=m.replace("[","")
    n=n.replace("]","")
    n=n.replace("'","")
    df.loc[i,'Zoom_link']=n

In [16]:
df.notnull().sum()

PostDate       689
Description    689
image_url      513
source_url     689
Zoom_link      689
start_time     540
event_title    540
dtype: int64

In [17]:
df.to_csv("Twitter_apr22_almost.csv",index=False)

In [18]:
import pandas as pd
final_df=pd.read_csv("C:\\Users\\Admin\\Twitter_apr22_almost.csv")
final_df

,PostDate,Description,image_url,source_url,Zoom_link,start_time,event_title
0,2021-04-22T06:53:04.000Z,Cheche\n@WinnieCheche\n·\n10s\nReplying to \n@...,NaN,https://twitter.com/WinnieCheche/status/138512...,NaN,NaN,NaN
1,2021-04-22T06:51:00.000Z,SITE_Africa\n@site_africa\n·\n2m\nIt's Earth D...,https://pbs.twimg.com/media/EzjyvydXEAATEhM?fo...,https://twitter.com/site_africa/status/1385123...,https://us02web.zoom.us/meeting/register/tZYlf...,"Apr 22, 2021 11:00 AM in Johannesburg",EARTH DAY SHOWCASE AFRICA
2,2021-04-22T06:51:00.000Z,Church of England Environment Programme\n@CofE...,NaN,https://twitter.com/CofEEnvironment/status/138...,https://zoom.us/meeting/register/tJYvc-2prT4tG...,"Apr 27, 2021 04:00 PM in London","Webinar on ""To replace or not replace?"" your c..."
3,2021-04-22T06:50:51.000Z,CefES Research Center\n@CefesCenter\n·\n2m\nOu...,https://pbs.twimg.com/media/EzjyyxuWYAE88Cs?fo...,https://twitter.com/CefesCenter/status/1385123...,NaN,NaN,NaN
4,2021-04-22T06:48:03.000Z,Monthly Review\n@monthly_review\n·\n5m\n#Trial...,https://pbs.twimg.com/media/EzjyOVqXoAITTK_?fo...,https://twitter.com/monthly_review/status/1385...,https://bit.ly/2RPmpVq,"Apr 23, 2021 12:00 PM in Central Time (US and ...","Virtual Book Launch: Michael Tigar's ""Sensing ..."
...,...,...,...,...,...,...,...
684,2021-04-21T13:42:16.000Z,Coach P\n@CoachP_Irish90\n·\n18h\nJoin us \n@N...,NaN,https://twitter.com/CoachP_Irish90/status/1384...,NaN,NaN,NaN
685,2021-04-21T13:34:27.000Z,AFP SouthEastOntario\n@AFP_SEO\n·\n18h\nJoin \...,https://pbs.twimg.com/media/EzgFoqlXoAAuIX-?fo...,https://twitter.com/AFP_SEO/status/13848631025...,https://us02web.zoom.us/meeting/register/tZwsc...,"Apr 28, 2021 12:00 PM in Eastern Time (US and ...",Smart Fundraising in Pandemic Times
686,2021-04-21T13:33:36.000Z,Teacher's Discovery\n@_TDSocial\n·\n18h\nJoin ...,https://pbs.twimg.com/media/EzgFSv0WUAAczvq?fo...,https://twitter.com/_TDSocial/status/138486288...,https://teachersdiscovery.zoom.us/meeting/regi...,"Apr 21, 2021 05:30 PM in Eastern Time (US and ...",A Free Interactive Presentation for World Lang...
687,2021-04-21T13:33:02.000Z,Saddam Sayyaleh\n@SaddamSayyaleh\n·\n18h\nIt's...,https://pbs.twimg.com/media/EzgFU3EXMAIPBRI?fo...,https://twitter.com/SaddamSayyaleh/status/1384...,https://us02web.zoom.us/meeting/register/tZAkc...,"Apr 22, 2021 04:00 PM in Amman",Youth Engagement during COVID-19 – Experiences...


In [19]:
final_df["start_time"]=final_df["start_time"].str.replace(" in",'',regex=True)

In [21]:
from datetime import datetime
from pytz import timezone
import time
import datetime
import pytz
import re
for iter1,j in enumerate(final_df["start_time"]):
    #March 2, 2021 10:00 AM 11:00 AM (PST)
    #pattern1=re.compile("\d*\s[A-Z][a-z]+\s\d*\s\s\d*:\d*")
    #May 27, 2021 8:00 PM
    print(j,iter1)
    if j== "nan":
        print("Hello")      
        continue
        
    pattern2=re.compile("[A-Z][a-z]+\s\d*,\s\d*\s\d*:\d*\s[A-Z]+")
    #pattern3=re.compile("[A-Z][a-z]+\s\d*,\s\d*\s\d*:\d*\s[A-Z]+")
    #pattern4=re.compile("\d*\s[A-Z][a-z]+\s\d*\s\s\d*:\d*\s[A-Z]+")
    #pattern5=re.compile("\d*\s[A-Z][a-z]+\s\d*\s\s\d*.\d*")
    #pattern6=re.compile("Mar\s\d*,\s\d*\s\d*:\d*\s[A-Z]+")
    #pattern7=re.compile("[A-Z][a-z]+\s\d*,\s\d*\s\d*:\d*\s[A-Z]+")
    #pattern8=re.compile("\d*\s[A-Z][a-z]+\s\d*\s\s\d*:\d*\s[a-z]+")
    #pattern9=re.compile("[A-Z][a-z]+\s\d*,\s\d*\s\s\d*[a-z]+")
    try:
        regex=pattern2.findall(j)
        print(regex,iter1)
        if regex==[]:
            print("E")
            continue
        regex_str=""
        regex_str=regex_str.join(regex)

        date_time_str = regex_str
    except:
        pass
    try:
        #date_time_obj = datetime.datetime.strptime(date_time_str, "%d %B %Y %H:%M")
        #1 March 2021 13:00
        date_time_obj=datetime.datetime.strptime(date_time_str,    "%b %d, %Y %I:%M %p")
        #March 2, 2021 10:00 AM
        print(date_time_obj,iter1)
        if "GMT" in j:
            timestamp = datetime.datetime.timestamp(date_time_obj)
            #updated_timestamp=""
            #updated_timestamp=updated_timestamp.join(timestamp)
            print(timestamp,iter1)
            final_df.at[iter1,"new_updated_timezone"]=timestamp
        elif ("EDT"or"EST"or"Eastern"or"ET") in j:
            print(iter1)
            old_timezone = pytz.timezone("US/Eastern")
            new_timezone = pytz.timezone("UTC")
            localized_timestamp = old_timezone.localize(date_time_obj)
            new_timezone_timestamp = localized_timestamp.astimezone(new_timezone)
            #ime_stamp=time.mktime(datetime.datetime.strptime(new_timezone_timestamp
            #.timetuple())
            print(timestamp)
            timestamp = datetime.datetime.timestamp(new_timezone_timestamp)
            final_df.at[iter1,"new_updated_timezone"]=timestamp
            print(final_df.at[iter1,"new_updated_timezone"])
        elif ("PST"or"Pacific"or"PDT"):
            old_timezone = pytz.timezone("US/Pacific")
            new_timezone = pytz.timezone("UTC")
            localized_timestamp = old_timezone.localize(date_time_obj)
            new_timezone_timestamp = localized_timestamp.astimezone(new_timezone)
            #ime_stamp=time.mktime(datetime.datetime.strptime(new_timezone_timestamp
            #.timetuple())
            timestamp = datetime.datetime.timestamp(new_timezone_timestamp)
            final_df.at[iter1,"new_updated_timezone"]=timestamp
        elif ("CST"or"Central"or"CENTRAL"or"CDT") :
            old_timezone = pytz.timezone("US/Central")
            new_timezone = pytz.timezone("UTC")
            localized_timestamp = old_timezone.localize(date_time_obj)
            new_timezone_timestamp = localized_timestamp.astimezone(new_timezone)
            #ime_stamp=time.mktime(datetime.datetime.strptime(new_timezone_timestamp
            #.timetuple())
            timestamp = datetime.datetime.timestamp(new_timezone_timestamp)
            final_df.at[iter1,"new_updated_timezone"]=timestamp
        elif("AEDT") in j :
            old_timezone = pytz.timezone("Australia/Sydney")
            new_timezone = pytz.timezone("UTC")
            localized_timestamp = old_timezone.localize(date_time_obj)
            new_timezone_timestamp = localized_timestamp.astimezone(new_timezone)
            #ime_stamp=time.mktime(datetime.datetime.strptime(new_timezone_timestamp
            #.timetuple())
            timestamp = datetime.datetime.timestamp(new_timezone_timestamp)
            final_df.at[iter1,"new_updated_timezone"]=timestamp
        elif ("ACDT") in j:
            old_timezone = pytz.timezone("Australia/Adelaide")
            new_timezone = pytz.timezone("UTC")
            localized_timestamp = old_timezone.localize(date_time_obj)
            new_timezone_timestamp = localized_timestamp.astimezone(new_timezone)
            #ime_stamp=time.mktime(datetime.datetime.strptime(new_timezone_timestamp
            #.timetuple())
            timestamp = datetime.datetime.timestamp(new_timezone_timestamp)
            final_df.at[iter1,"new_updated_timezone"]=timestamp
        elif ("JST") in j:
            old_timezone = pytz.timezone("Japan")
            new_timezone = pytz.timezone("UTC")
            localized_timestamp = old_timezone.localize(date_time_obj)
            new_timezone_timestamp = localized_timestamp.astimezone(new_timezone)
            #ime_stamp=time.mktime(datetime.datetime.strptime(new_timezone_timestamp
            #.timetuple())
            timestamp = datetime.datetime.timestamp(new_timezone_timestamp)
            final_df.at[iter1,"new_updated_timezone"]=timestamp
        elif ("MST"or"Mountain") in j:
            old_timezone = pytz.timezone("MST")
            new_timezone = pytz.timezone("UTC")
            localized_timestamp = old_timezone.localize(date_time_obj)
            new_timezone_timestamp = localized_timestamp.astimezone(new_timezone)
            #ime_stamp=time.mktime(datetime.datetime.strptime(new_timezone_timestamp
            #.timetuple())
            timestamp = datetime.datetime.timestamp(new_timezone_timestamp)
            final_df.at[iter1,"new_updated_timezone"]=timestamp
        elif("EET") in j:
            old_timezone = pytz.timezone("EET")
            new_timezone = pytz.timezone("UTC")
            localized_timestamp = old_timezone.localize(date_time_obj)
            new_timezone_timestamp = localized_timestamp.astimezone(new_timezone)
            #ime_stamp=time.mktime(datetime.datetime.strptime(new_timezone_timestamp
            #.timetuple())
            timestamp = datetime.datetime.timestamp(new_timezone_timestamp)
            final_df.at[iter1,"new_updated_timezone"]=timestamp
        elif ("CET") in j:
            old_timezone = pytz.timezone("CET")
            new_timezone = pytz.timezone("UTC")
            localized_timestamp = old_timezone.localize(date_time_obj)
            new_timezone_timestamp = localized_timestamp.astimezone(new_timezone)
            #ime_stamp=time.mktime(datetime.datetime.strptime(new_timezone_timestamp
            #.timetuple())
            timestamp = datetime.datetime.timestamp(new_timezone_timestamp)
            final_df.at[iter1,"new_updated_timezone"]=timestamp
        elif ("Hawaii") in j:
            old_timezone = pytz.timezone("US/Hawaii")
            new_timezone = pytz.timezone("UTC")
            localized_timestamp = old_timezone.localize(date_time_obj)
            new_timezone_timestamp = localized_timestamp.astimezone(new_timezone)
            #ime_stamp=time.mktime(datetime.datetime.strptime(new_timezone_timestamp
            #.timetuple())
            timestamp = datetime.datetime.timestamp(new_timezone_timestamp)
            final_df.at[iter1,"new_updated_timezone"]=timestamp
        elif ("India") in j:
            old_timezone = pytz.timezone("Asia/Kolkata")
            new_timezone = pytz.timezone("UTC")
            localized_timestamp = old_timezone.localize(date_time_obj)
            new_timezone_timestamp = localized_timestamp.astimezone(new_timezone)
            #ime_stamp=time.mktime(datetime.datetime.strptime(new_timezone_timestamp
            #.timetuple())
            timestamp = datetime.datetime.timestamp(new_timezone_timestamp)
            final_df.at[iter1,"new_updated_timezone"]=timestamp
        elif ("Indiana") in j:
            old_timezone = pytz.timezone("US/East-Indiana")
            new_timezone = pytz.timezone("UTC")
            localized_timestamp = old_timezone.localize(date_time_obj)
            new_timezone_timestamp = localized_timestamp.astimezone(new_timezone)
            #ime_stamp=time.mktime(datetime.datetime.strptime(new_timezone_timestamp
            #.timetuple())
            timestamp = datetime.datetime.timestamp(new_timezone_timestamp)
            final_df.at[iter1,"new_updated_timezone"]=timestamp
        elif ("Dubai") in j:
            old_timezone = pytz.timezone("Asia/Dubai")
            new_timezone = pytz.timezone("UTC")
            localized_timestamp = old_timezone.localize(date_time_obj)
            new_timezone_timestamp = localized_timestamp.astimezone(new_timezone)
            #ime_stamp=time.mktime(datetime.datetime.strptime(new_timezone_timestamp
            #.timetuple())
            timestamp = datetime.datetime.timestamp(new_timezone_timestamp)
            final_df.at[iter1,"new_updated_timezone"]=timestamp
        elif ("London") in j:
            old_timezone = pytz.timezone("Europe/London")
            new_timezone = pytz.timezone("UTC")
            localized_timestamp = old_timezone.localize(date_time_obj)
            new_timezone_timestamp = localized_timestamp.astimezone(new_timezone)
            #ime_stamp=time.mktime(datetime.datetime.strptime(new_timezone_timestamp
            #.timetuple())
            timestamp = datetime.datetime.timestamp(new_timezone_timestamp)
            final_df.at[iter1,"new_updated_timezone"]=timestamp
        elif ("Bangkok") in j:
            old_timezone = pytz.timezone("Asia/Bangkok")
            new_timezone = pytz.timezone("UTC")
            localized_timestamp = old_timezone.localize(date_time_obj)
            new_timezone_timestamp = localized_timestamp.astimezone(new_timezone)
            #ime_stamp=time.mktime(datetime.datetime.strptime(new_timezone_timestamp
            #.timetuple())
            timestamp = datetime.datetime.timestamp(new_timezone_timestamp)
            final_df.at[iter1,"new_updated_timezone"]=timestamp
        elif ("Jakarta") in j:
            old_timezone = pytz.timezone("Asia/Jakarta")
            new_timezone = pytz.timezone("UTC")
            localized_timestamp = old_timezone.localize(date_time_obj)
            new_timezone_timestamp = localized_timestamp.astimezone(new_timezone)
            #ime_stamp=time.mktime(datetime.datetime.strptime(new_timezone_timestamp
            #.timetuple())
            timestamp = datetime.datetime.timestamp(new_timezone_timestamp)
            final_df.at[iter1,"new_updated_timezone"]=timestamp
        elif ("Manila") in j:
            old_timezone = pytz.timezone("Asia/Manila")
            new_timezone = pytz.timezone("UTC")
            localized_timestamp = old_timezone.localize(date_time_obj)
            new_timezone_timestamp = localized_timestamp.astimezone(new_timezone)
            #ime_stamp=time.mktime(datetime.datetime.strptime(new_timezone_timestamp
            #.timetuple())
            timestamp = datetime.datetime.timestamp(new_timezone_timestamp)
            final_df.at[iter1,"new_updated_timezone"]=timestamp
        elif ("Singapore") in j:
            old_timezone = pytz.timezone("Asia/Singapore")
            new_timezone = pytz.timezone("UTC")
            localized_timestamp = old_timezone.localize(date_time_obj)
            new_timezone_timestamp = localized_timestamp.astimezone(new_timezone)
            #ime_stamp=time.mktime(datetime.datetime.strptime(new_timezone_timestamp
            #.timetuple())
            timestamp = datetime.datetime.timestamp(new_timezone_timestamp)
            final_df.at[iter1,"new_updated_timezone"]=timestamp
        elif ("Taipei") in j:
            old_timezone = pytz.timezone("Asia/Taipei")
            new_timezone = pytz.timezone("UTC")
            localized_timestamp = old_timezone.localize(date_time_obj)
            new_timezone_timestamp = localized_timestamp.astimezone(new_timezone)
            #ime_stamp=time.mktime(datetime.datetime.strptime(new_timezone_timestamp
            #.timetuple())
            timestamp = datetime.datetime.timestamp(new_timezone_timestamp)
            final_df.at[iter1,"new_updated_timezone"]=timestamp
        elif ("Karachi") in j:
            old_timezone = pytz.timezone("Asia/Karachi")
            new_timezone = pytz.timezone("UTC")
            localized_timestamp = old_timezone.localize(date_time_obj)
            new_timezone_timestamp = localized_timestamp.astimezone(new_timezone)
            #ime_stamp=time.mktime(datetime.datetime.strptime(new_timezone_timestamp
            #.timetuple())
            timestamp = datetime.datetime.timestamp(new_timezone_timestamp)
            final_df.at[iter1,"new_updated_timezone"]=timestamp
        elif ("Hong_Kong") in j:
            old_timezone = pytz.timezone("Asia/Hong_Kong")
            new_timezone = pytz.timezone("UTC")
            localized_timestamp = old_timezone.localize(date_time_obj)
            new_timezone_timestamp = localized_timestamp.astimezone(new_timezone)
            #ime_stamp=time.mktime(datetime.datetime.strptime(new_timezone_timestamp
            #.timetuple())
            timestamp = datetime.datetime.timestamp(new_timezone_timestamp)
            final_df.at[iter1,"new_updated_timezone"]=timestamp
        elif ("Riyadh") in j:
            old_timezone = pytz.timezone("Asia/Riyadh")
            new_timezone = pytz.timezone("UTC")
            localized_timestamp = old_timezone.localize(date_time_obj)
            new_timezone_timestamp = localized_timestamp.astimezone(new_timezone)
            #ime_stamp=time.mktime(datetime.datetime.strptime(new_timezone_timestamp
            #.timetuple())
            timestamp = datetime.datetime.timestamp(new_timezone_timestamp)
            final_df.at[iter1,"new_updated_timezone"]=timestamp
        elif ("Auckland") in j:
            old_timezone = pytz.timezone("NZ")
            new_timezone = pytz.timezone("UTC")
            localized_timestamp = old_timezone.localize(date_time_obj)
            new_timezone_timestamp = localized_timestamp.astimezone(new_timezone)
            #ime_stamp=time.mktime(datetime.datetime.strptime(new_timezone_timestamp
            #.timetuple())
            timestamp = datetime.datetime.timestamp(new_timezone_timestamp)
            final_df.at[iter1,"new_updated_timezone"]=timestamp
        elif ("Harare") in j:
            old_timezone = pytz.timezone("Africa/Harare")
            new_timezone = pytz.timezone("UTC")
            localized_timestamp = old_timezone.localize(date_time_obj)
            new_timezone_timestamp = localized_timestamp.astimezone(new_timezone)
            #ime_stamp=time.mktime(datetime.datetime.strptime(new_timezone_timestamp
            #.timetuple())
            timestamp = datetime.datetime.timestamp(new_timezone_timestamp)
            final_df.at[iter1,"new_updated_timezone"]=timestamp
        elif ("Mountreal") in j:
            old_timezone = pytz.timezone("America/Montreal")
            new_timezone = pytz.timezone("UTC")
            localized_timestamp = old_timezone.localize(date_time_obj)
            new_timezone_timestamp = localized_timestamp.astimezone(new_timezone)
            #ime_stamp=time.mktime(datetime.datetime.strptime(new_timezone_timestamp
            #.timetuple())
            timestamp = datetime.datetime.timestamp(new_timezone_timestamp)
            final_df.at[iter1,"new_updated_timezone"]=timestamp
        elif ("Adelaide") in j:
            old_timezone = pytz.timezone("Australia/Adelaide")
            new_timezone = pytz.timezone("UTC")
            localized_timestamp = old_timezone.localize(date_time_obj)
            new_timezone_timestamp = localized_timestamp.astimezone(new_timezone)
            #ime_stamp=time.mktime(datetime.datetime.strptime(new_timezone_timestamp
            #.timetuple())
            timestamp = datetime.datetime.timestamp(new_timezone_timestamp)
            final_df.at[iter1,"new_updated_timezone"]=timestamp
        elif ("Edmonton") in j:
            old_timezone = pytz.timezone("America/Edmonton")
            new_timezone = pytz.timezone("UTC")
            localized_timestamp = old_timezone.localize(date_time_obj)
            new_timezone_timestamp = localized_timestamp.astimezone(new_timezone)
            #ime_stamp=time.mktime(datetime.datetime.strptime(new_timezone_timestamp
            #.timetuple())
            timestamp = datetime.datetime.timestamp(new_timezone_timestamp)
            final_df.at[iter1,"new_updated_timezone"]=timestamp
        elif ("Bairut") in j:
            old_timezone = pytz.timezone("Asia/Beirut")
            new_timezone = pytz.timezone("UTC")
            localized_timestamp = old_timezone.localize(date_time_obj)
            new_timezone_timestamp = localized_timestamp.astimezone(new_timezone)
            #ime_stamp=time.mktime(datetime.datetime.strptime(new_timezone_timestamp
            #.timetuple())
            timestamp = datetime.datetime.timestamp(new_timezone_timestamp)
            final_df.at[iter1,"new_updated_timezone"]=timestamp
        elif ("Kuwait") in j:
            old_timezone = pytz.timezone("Asia/Kuwait")
            new_timezone = pytz.timezone("UTC")
            localized_timestamp = old_timezone.localize(date_time_obj)
            new_timezone_timestamp = localized_timestamp.astimezone(new_timezone)
            #ime_stamp=time.mktime(datetime.datetime.strptime(new_timezone_timestamp
            #.timetuple())
            timestamp = datetime.datetime.timestamp(new_timezone_timestamp)
            final_df.at[iter1,"new_updated_timezone"]=timestamp
        elif ("Kiev") in j:
            old_timezone = pytz.timezone("Europe/Kiev")
            new_timezone = pytz.timezone("UTC")
            localized_timestamp = old_timezone.localize(date_time_obj)
            new_timezone_timestamp = localized_timestamp.astimezone(new_timezone)
            #ime_stamp=time.mktime(datetime.datetime.strptime(new_timezone_timestamp
            #.timetuple())
            timestamp = datetime.datetime.timestamp(new_timezone_timestamp)
            final_df.at[iter1,"new_updated_timezone"]=timestamp
        elif ("Dublin") in j:
            old_timezone = pytz.timezone("Europe/Dublin")
            new_timezone = pytz.timezone("UTC")
            localized_timestamp = old_timezone.localize(date_time_obj)
            new_timezone_timestamp = localized_timestamp.astimezone(new_timezone)
            #ime_stamp=time.mktime(datetime.datetime.strptime(new_timezone_timestamp
            #.timetuple())
            timestamp = datetime.datetime.timestamp(new_timezone_timestamp)
            final_df.at[iter1,"new_updated_timezone"]=timestamp
        elif ("Bahrain") in j:
            old_timezone = pytz.timezone("Asia/Bahrain")
            new_timezone = pytz.timezone("UTC")
            localized_timestamp = old_timezone.localize(date_time_obj)
            new_timezone_timestamp = localized_timestamp.astimezone(new_timezone)
            #ime_stamp=time.mktime(datetime.datetime.strptime(new_timezone_timestamp
            #.timetuple())
            timestamp = datetime.datetime.timestamp(new_timezone_timestamp)
            final_df.at[iter1,"new_updated_timezone"]=timestamp
        elif ("Baku") in j:
            old_timezone = pytz.timezone("Asia/Baku")
            new_timezone = pytz.timezone("UTC")
            localized_timestamp = old_timezone.localize(date_time_obj)
            new_timezone_timestamp = localized_timestamp.astimezone(new_timezone)
            #ime_stamp=time.mktime(datetime.datetime.strptime(new_timezone_timestamp
            #.timetuple())
            timestamp = datetime.datetime.timestamp(new_timezone_timestamp)
            final_df.at[iter1,"new_updated_timezone"]=timestamp
        elif ("Muscat") in j:
            old_timezone = pytz.timezone("Asia/Muscat")
            new_timezone = pytz.timezone("UTC")
            localized_timestamp = old_timezone.localize(date_time_obj)
            new_timezone_timestamp = localized_timestamp.astimezone(new_timezone)
            #ime_stamp=time.mktime(datetime.datetime.strptime(new_timezone_timestamp
            #.timetuple())
            timestamp = datetime.datetime.timestamp(new_timezone_timestamp)
            final_df.at[iter1,"new_updated_timezone"]=timestamp
        elif ("Santiago") in j:
            old_timezone = pytz.timezone("America/Santiago")
            new_timezone = pytz.timezone("UTC")
            localized_timestamp = old_timezone.localize(date_time_obj)
            new_timezone_timestamp = localized_timestamp.astimezone(new_timezone)
            #ime_stamp=time.mktime(datetime.datetime.strptime(new_timezone_timestamp
            #.timetuple())
            timestamp = datetime.datetime.timestamp(new_timezone_timestamp)
            final_df.at[iter1,"new_updated_timezone"]=timestamp
        elif ("Santiago") in j:
            old_timezone = pytz.timezone("Australia/Perth")
            new_timezone = pytz.timezone("UTC")
            localized_timestamp = old_timezone.localize(date_time_obj)
            new_timezone_timestamp = localized_timestamp.astimezone(new_timezone)
            #ime_stamp=time.mktime(datetime.datetime.strptime(new_timezone_timestamp
            #.timetuple())
            timestamp = datetime.datetime.timestamp(new_timezone_timestamp)
            final_df.at[iter1,"new_updated_timezone"]=timestamp
        elif ("Halifax") in j:
            old_timezone = pytz.timezone("America/Halifax")
            new_timezone = pytz.timezone("UTC")
            localized_timestamp = old_timezone.localize(date_time_obj)
            new_timezone_timestamp = localized_timestamp.astimezone(new_timezone)
            #ime_stamp=time.mktime(datetime.datetime.strptime(new_timezone_timestamp
            #.timetuple())
            timestamp = datetime.datetime.timestamp(new_timezone_timestamp)
            final_df.at[iter1,"new_updated_timezone"]=timestamp
        elif ("Casablanca") in j:
            old_timezone = pytz.timezone("Africa/Casablanca")
            new_timezone = pytz.timezone("UTC")
            localized_timestamp = old_timezone.localize(date_time_obj)
            new_timezone_timestamp = localized_timestamp.astimezone(new_timezone)
            #ime_stamp=time.mktime(datetime.datetime.strptime(new_timezone_timestamp
            #.timetuple())
            timestamp = datetime.datetime.timestamp(new_timezone_timestamp)
            final_df.at[iter1,"new_updated_timezone"]=timestamp
        elif ("La_Paz") in j:
            old_timezone = pytz.timezone("America/La_Paz")
            new_timezone = pytz.timezone("UTC")
            localized_timestamp = old_timezone.localize(date_time_obj)
            new_timezone_timestamp = localized_timestamp.astimezone(new_timezone)
            #ime_stamp=time.mktime(datetime.datetime.strptime(new_timezone_timestamp
            #.timetuple())
            timestamp = datetime.datetime.timestamp(new_timezone_timestamp)
            final_df.at[iter1,"new_updated_timezone"]=timestamp
        elif ("Guyana") in j:
            old_timezone = pytz.timezone("America/Guyana")
            new_timezone = pytz.timezone("UTC")
            localized_timestamp = old_timezone.localize(date_time_obj)
            new_timezone_timestamp = localized_timestamp.astimezone(new_timezone)
            #ime_stamp=time.mktime(datetime.datetime.strptime(new_timezone_timestamp
            #.timetuple())
            timestamp = datetime.datetime.timestamp(new_timezone_timestamp)
            final_df.at[iter1,"new_updated_timezone"]=timestamp
        elif ("Jerusalem") in j:
            old_timezone = pytz.timezone("Asia/Jerusalem")
            new_timezone = pytz.timezone("UTC")
            localized_timestamp = old_timezone.localize(date_time_obj)
            new_timezone_timestamp = localized_timestamp.astimezone(new_timezone)
            #ime_stamp=time.mktime(datetime.datetime.strptime(new_timezone_timestamp
            #.timetuple())
            timestamp = datetime.datetime.timestamp(new_timezone_timestamp)
            final_df.at[iter1,"new_updated_timezone"]=timestamp
        elif ("Almaty") in j:
            old_timezone = pytz.timezone("Asia/Almaty")
            new_timezone = pytz.timezone("UTC")
            localized_timestamp = old_timezone.localize(date_time_obj)
            new_timezone_timestamp = localized_timestamp.astimezone(new_timezone)
            #ime_stamp=time.mktime(datetime.datetime.strptime(new_timezone_timestamp
            #.timetuple())
            timestamp = datetime.datetime.timestamp(new_timezone_timestamp)
            final_df.at[iter1,"new_updated_timezone"]=timestamp
        elif ("Caracas") in j:
            old_timezone = pytz.timezone("America/Caracas")
            new_timezone = pytz.timezone("UTC")
            localized_timestamp = old_timezone.localize(date_time_obj)
            new_timezone_timestamp = localized_timestamp.astimezone(new_timezone)
            #ime_stamp=time.mktime(datetime.datetime.strptime(new_timezone_timestamp
            #.timetuple())
            timestamp = datetime.datetime.timestamp(new_timezone_timestamp)
            final_df.at[iter1,"new_updated_timezone"]=timestamp
        elif ("Lima") in j:
            old_timezone = pytz.timezone("America/Lima")
            new_timezone = pytz.timezone("UTC")
            localized_timestamp = old_timezone.localize(date_time_obj)
            new_timezone_timestamp = localized_timestamp.astimezone(new_timezone)
            #ime_stamp=time.mktime(datetime.datetime.strptime(new_timezone_timestamp
            #.timetuple())
            timestamp = datetime.datetime.timestamp(new_timezone_timestamp)
            final_df.at[iter1,"new_updated_timezone"]=timestamp
        elif ("Lisbon") in j:
            old_timezone = pytz.timezone("Europe/Lisbon")
            new_timezone = pytz.timezone("UTC")
            localized_timestamp = old_timezone.localize(date_time_obj)
            new_timezone_timestamp = localized_timestamp.astimezone(new_timezone)
            #ime_stamp=time.mktime(datetime.datetime.strptime(new_timezone_timestamp
            #.timetuple())
            timestamp = datetime.datetime.timestamp(new_timezone_timestamp)
            final_df.at[iter1,"new_updated_timezone"]=timestamp
        elif ("Puerto_Rico") in j:
            old_timezone = pytz.timezone("America/Puerto_Rico")
            new_timezone = pytz.timezone("UTC")
            localized_timestamp = old_timezone.localize(date_time_obj)
            new_timezone_timestamp = localized_timestamp.astimezone(new_timezone)
            #ime_stamp=time.mktime(datetime.datetime.strptime(new_timezone_timestamp
            #.timetuple())
            timestamp = datetime.datetime.timestamp(new_timezone_timestamp)
            final_df.at[iter1,"new_updated_timezone"]=timestamp
        elif ("Muscat") in j:
            old_timezone = pytz.timezone("Asia/Muscat")
            new_timezone = pytz.timezone("UTC")
            localized_timestamp = old_timezone.localize(date_time_obj)
            new_timezone_timestamp = localized_timestamp.astimezone(new_timezone)
            #ime_stamp=time.mktime(datetime.datetime.strptime(new_timezone_timestamp
            #.timetuple())
            timestamp = datetime.datetime.timestamp(new_timezone_timestamp)
            final_df.at[iter1,"new_updated_timezone"]=timestamp
        elif ("Guyana") in j:
            old_timezone = pytz.timezone("America/Guyana")
            new_timezone = pytz.timezone("UTC")
            localized_timestamp = old_timezone.localize(date_time_obj)
            new_timezone_timestamp = localized_timestamp.astimezone(new_timezone)
            #ime_stamp=time.mktime(datetime.datetime.strptime(new_timezone_timestamp
            #.timetuple())
            timestamp = datetime.datetime.timestamp(new_timezone_timestamp)
            final_df.at[iter1,"new_updated_timezone"]=timestamp
        date_time_str=""
        
    except:
        pass

nan 0
Apr 22, 2021 11:00 AM Johannesburg 1
['Apr 22, 2021 11:00 AM'] 1
2021-04-22 11:00:00 1
Apr 27, 2021 04:00 PM London 2
['Apr 27, 2021 04:00 PM'] 2
2021-04-27 16:00:00 2
nan 3
Apr 23, 2021 12:00 PM Central Time (US and Canada) 4
['Apr 23, 2021 12:00 PM'] 4
2021-04-23 12:00:00 4
May 3, 2021 10:00 AM Johannesburg 5
['May 3, 2021 10:00 AM'] 5
2021-05-03 10:00:00 5
Time shows West Central Africa 6
[] 6
E
Apr 22, 2021 02:00 PM Taipei 7
['Apr 22, 2021 02:00 PM'] 7
2021-04-22 14:00:00 7
Apr 23, 2021 03:00 PM Kuala Lumpur 8
['Apr 23, 2021 03:00 PM'] 8
2021-04-23 15:00:00 8
May 15, 2021 01:30 PM London 9
['May 15, 2021 01:30 PM'] 9
2021-05-15 13:30:00 9
nan 10
Apr 25, 2021 06:00 PM West Central Africa 11
['Apr 25, 2021 06:00 PM'] 11
2021-04-25 18:00:00 11
Apr 22, 2021 01:00 PM Eastern Time (US and Canada) 12
['Apr 22, 2021 01:00 PM'] 12
2021-04-22 13:00:00 12
nan 13
Time shows Canberra, Melbourne, Sydney 14
[] 14
E
Apr 22, 2021 02:00 PM London 15
['Apr 22, 2021 02:00 PM'] 15
2021-04-22 14:0

E
nan 233
Apr 28, 2021 12:00 PM Eastern Time (US and Canada) 234
['Apr 28, 2021 12:00 PM'] 234
2021-04-28 12:00:00 234
Apr 22, 2021 01:00 PM Eastern Time (US and Canada) 235
['Apr 22, 2021 01:00 PM'] 235
2021-04-22 13:00:00 235
Time shows Eastern Time (US and Canada) 236
[] 236
E
Apr 30, 2021 10:00 AM Eastern Time (US and Canada) 237
['Apr 30, 2021 10:00 AM'] 237
2021-04-30 10:00:00 237
nan 238
Apr 22, 2021 01:00 PM Eastern Time (US and Canada) 239
['Apr 22, 2021 01:00 PM'] 239
2021-04-22 13:00:00 239
Apr 26, 2021 09:00 AM Central Time (US and Canada) 240
['Apr 26, 2021 09:00 AM'] 240
2021-04-26 09:00:00 240
nan 241
Apr 22, 2021 06:00 PM Eastern Time (US and Canada) 242
['Apr 22, 2021 06:00 PM'] 242
2021-04-22 18:00:00 242
Apr 29, 2021 07:00 PM London 243
['Apr 29, 2021 07:00 PM'] 243
2021-04-29 19:00:00 243
nan 244
Apr 29, 2021 01:00 PM Eastern Time (US and Canada) 245
['Apr 29, 2021 01:00 PM'] 245
2021-04-29 13:00:00 245
Time shows Eastern Time (US and Canada) 246
[] 246
E
Apr 22, 20

2021-04-22 11:00:00 353
nan 354
Apr 27, 2021 11:00 AM Eastern Time (US and Canada) 355
['Apr 27, 2021 11:00 AM'] 355
2021-04-27 11:00:00 355
Apr 24, 2021 10:00 AM Pacific Time (US and Canada) 356
['Apr 24, 2021 10:00 AM'] 356
2021-04-24 10:00:00 356
Apr 22, 2021 06:00 PM Mountain Time (US and Canada) 357
['Apr 22, 2021 06:00 PM'] 357
2021-04-22 18:00:00 357
May 12, 2021 10:00 AM Pacific Time (US and Canada) 358
['May 12, 2021 10:00 AM'] 358
2021-05-12 10:00:00 358
Apr 27, 2021 12:00 PM Pacific Time (US and Canada) 359
['Apr 27, 2021 12:00 PM'] 359
2021-04-27 12:00:00 359
Apr 22, 2021 12:00 PM Eastern Time (US and Canada) 360
['Apr 22, 2021 12:00 PM'] 360
2021-04-22 12:00:00 360
Apr 21, 2021 07:00 PM Eastern Time (US and Canada) 361
['Apr 21, 2021 07:00 PM'] 361
2021-04-21 19:00:00 361
Apr 23, 2021 12:00 PM Eastern Time (US and Canada) 362
['Apr 23, 2021 12:00 PM'] 362
2021-04-23 12:00:00 362
Apr 22, 2021 03:00 PM Eastern Time (US and Canada) 363
['Apr 22, 2021 03:00 PM'] 363
2021-04-22

['Apr 22, 2021 01:00 PM'] 547
2021-04-22 13:00:00 547
May 6, 2021 06:00 PM 548
['May 6, 2021 06:00 PM'] 548
2021-05-06 18:00:00 548
Apr 22, 2021 10:00 AM Central Time (US and Canada) 549
['Apr 22, 2021 10:00 AM'] 549
2021-04-22 10:00:00 549
Apr 22, 2021 06:00 PM Eastern Time (US and Canada) 550
['Apr 22, 2021 06:00 PM'] 550
2021-04-22 18:00:00 550
Apr 21, 2021 05:00 PM Central Time (US and Canada) 551
['Apr 21, 2021 05:00 PM'] 551
2021-04-21 17:00:00 551
nan 552
Apr 22, 2021 03:30 PM Eastern Time (US and Canada) 553
['Apr 22, 2021 03:30 PM'] 553
2021-04-22 15:30:00 553
May 5, 2021 12:00 PM Eastern Time (US and Canada) 554
['May 5, 2021 12:00 PM'] 554
2021-05-05 12:00:00 554
Apr 28, 2021 04:00 PM Mexico City 555
['Apr 28, 2021 04:00 PM'] 555
2021-04-28 16:00:00 555
Stay signed 556
[] 556
E
Apr 28, 2021 08:00 PM Dublin 557
['Apr 28, 2021 08:00 PM'] 557
2021-04-28 20:00:00 557
Apr 22, 2021 05:00 PM Mexico City 558
['Apr 22, 2021 05:00 PM'] 558
2021-04-22 17:00:00 558
May 5, 2021 07:00 PM 

In [22]:
for i,j in enumerate(final_df['start_time']):
    print(i,j)

0 nan
1 Apr 22, 2021 11:00 AM Johannesburg
2 Apr 27, 2021 04:00 PM London
3 nan
4 Apr 23, 2021 12:00 PM Central Time (US and Canada)
5 May 3, 2021 10:00 AM Johannesburg
6 Time shows West Central Africa
7 Apr 22, 2021 02:00 PM Taipei
8 Apr 23, 2021 03:00 PM Kuala Lumpur
9 May 15, 2021 01:30 PM London
10 nan
11 Apr 25, 2021 06:00 PM West Central Africa
12 Apr 22, 2021 01:00 PM Eastern Time (US and Canada)
13 nan
14 Time shows Canberra, Melbourne, Sydney
15 Apr 22, 2021 02:00 PM London
16 Apr 22, 2021 09:30 PM Dubai
17 Apr 22, 2021 07:00 PM Mountain Time (US and Canada)
18 nan
19 Apr 22, 2021 07:30 PM Nairobi
20 nan
21 Apr 22, 2021 10:00 AM Nairobi
22 nan
23 Apr 23, 2021 07:00 PM India
24 Apr 23, 2021 08:00 PM India
25 May 6, 2021 04:00 PM Beijing, Shanghai
26 nan
27 Apr 28, 2021 06:00 PM Pacific Time (US and Canada)
28 Apr 27, 2021 02:00 PM Pacific Time (US and Canada)
29 nan
30 Apr 26, 2021 01:30 PM Singapore
31 nan
32 Apr 29, 2021 05:30 PM Pacific Time (US and Canada)
33 Apr 24, 2021 0

In [33]:
import pytz
pytz.all_timezones

['Africa/Abidjan', 'Africa/Accra', 'Africa/Addis_Ababa', 'Africa/Algiers', 'Africa/Asmara', 'Africa/Asmera', 'Africa/Bamako', 'Africa/Bangui', 'Africa/Banjul', 'Africa/Bissau', 'Africa/Blantyre', 'Africa/Brazzaville', 'Africa/Bujumbura', 'Africa/Cairo', 'Africa/Casablanca', 'Africa/Ceuta', 'Africa/Conakry', 'Africa/Dakar', 'Africa/Dar_es_Salaam', 'Africa/Djibouti', 'Africa/Douala', 'Africa/El_Aaiun', 'Africa/Freetown', 'Africa/Gaborone', 'Africa/Harare', 'Africa/Johannesburg', 'Africa/Juba', 'Africa/Kampala', 'Africa/Khartoum', 'Africa/Kigali', 'Africa/Kinshasa', 'Africa/Lagos', 'Africa/Libreville', 'Africa/Lome', 'Africa/Luanda', 'Africa/Lubumbashi', 'Africa/Lusaka', 'Africa/Malabo', 'Africa/Maputo', 'Africa/Maseru', 'Africa/Mbabane', 'Africa/Mogadishu', 'Africa/Monrovia', 'Africa/Nairobi', 'Africa/Ndjamena', 'Africa/Niamey', 'Africa/Nouakchott', 'Africa/Ouagadougou', 'Africa/Porto-Novo', 'Africa/Sao_Tome', 'Africa/Timbuktu', 'Africa/Tripoli', 'Africa/Tunis', 'Africa/Windhoek', 'Ameri

In [23]:
for i ,j in   enumerate(final_df['new_updated_timezone']):
    print(i,j)

0 nan
1 1619114400.0
2 1619564400.0
3 nan
4 1619204400.0
5 1620061200.0
6 nan
7 1619125200.0
8 1619215200.0
9 1621110600.0
10 nan
11 1619398800.0
12 1619121600.0
13 nan
14 nan
15 1619125200.0
16 1619152200.0
17 1619143200.0
18 nan
19 1619145000.0
20 nan
21 1619110800.0
22 nan
23 1619229600.0
24 1619233200.0
25 1620342000.0
26 nan
27 1619658000.0
28 1619557200.0
29 nan
30 1619469000.0
31 nan
32 1619742600.0
33 1619280000.0
34 nan
35 1619744400.0
36 nan
37 1619132400.0
38 nan
39 nan
40 nan
41 nan
42 1619312400.0
43 1619298000.0
44 nan
45 nan
46 1619208000.0
47 1619114400.0
48 1621548000.0
49 nan
50 nan
51 1619636400.0
52 1619224200.0
53 nan
54 1619218800.0
55 1619739000.0
56 nan
57 1619560800.0
58 1619110800.0
59 1619112600.0
60 1619118000.0
61 1619118000.0
62 1619146800.0
63 1620698400.0
64 nan
65 nan
66 nan
67 1619744400.0
68 1619145000.0
69 1621558800.0
70 nan
71 1619060400.0
72 1619110800.0
73 1619636400.0
74 nan
75 1619137800.0
76 1619053200.0
77 nan
78 1619053200.0
79 1619114400.0


In [24]:
final_df.notnull().sum()

PostDate                689
Description             689
image_url               513
source_url              689
Zoom_link               551
start_time              525
event_title             540
new_updated_timezone    490
dtype: int64

In [25]:
final_df.to_csv('TwitterApr22_timestamp.csv',index=False)
final_df

,PostDate,Description,image_url,source_url,Zoom_link,start_time,event_title,new_updated_timezone
0,2021-04-22T06:53:04.000Z,Cheche\n@WinnieCheche\n·\n10s\nReplying to \n@...,NaN,https://twitter.com/WinnieCheche/status/138512...,NaN,NaN,NaN,NaN
1,2021-04-22T06:51:00.000Z,SITE_Africa\n@site_africa\n·\n2m\nIt's Earth D...,https://pbs.twimg.com/media/EzjyvydXEAATEhM?fo...,https://twitter.com/site_africa/status/1385123...,https://us02web.zoom.us/meeting/register/tZYlf...,"Apr 22, 2021 11:00 AM Johannesburg",EARTH DAY SHOWCASE AFRICA,1.619114e+09
2,2021-04-22T06:51:00.000Z,Church of England Environment Programme\n@CofE...,NaN,https://twitter.com/CofEEnvironment/status/138...,https://zoom.us/meeting/register/tJYvc-2prT4tG...,"Apr 27, 2021 04:00 PM London","Webinar on ""To replace or not replace?"" your c...",1.619564e+09
3,2021-04-22T06:50:51.000Z,CefES Research Center\n@CefesCenter\n·\n2m\nOu...,https://pbs.twimg.com/media/EzjyyxuWYAE88Cs?fo...,https://twitter.com/CefesCenter/status/1385123...,NaN,NaN,NaN,NaN
4,2021-04-22T06:48:03.000Z,Monthly Review\n@monthly_review\n·\n5m\n#Trial...,https://pbs.twimg.com/media/EzjyOVqXoAITTK_?fo...,https://twitter.com/monthly_review/status/1385...,https://bit.ly/2RPmpVq,"Apr 23, 2021 12:00 PM Central Time (US and Can...","Virtual Book Launch: Michael Tigar's ""Sensing ...",1.619204e+09
...,...,...,...,...,...,...,...,...
684,2021-04-21T13:42:16.000Z,Coach P\n@CoachP_Irish90\n·\n18h\nJoin us \n@N...,NaN,https://twitter.com/CoachP_Irish90/status/1384...,NaN,NaN,NaN,NaN
685,2021-04-21T13:34:27.000Z,AFP SouthEastOntario\n@AFP_SEO\n·\n18h\nJoin \...,https://pbs.twimg.com/media/EzgFoqlXoAAuIX-?fo...,https://twitter.com/AFP_SEO/status/13848631025...,https://us02web.zoom.us/meeting/register/tZwsc...,"Apr 28, 2021 12:00 PM Eastern Time (US and Can...",Smart Fundraising in Pandemic Times,1.619636e+09
686,2021-04-21T13:33:36.000Z,Teacher's Discovery\n@_TDSocial\n·\n18h\nJoin ...,https://pbs.twimg.com/media/EzgFSv0WUAAczvq?fo...,https://twitter.com/_TDSocial/status/138486288...,https://teachersdiscovery.zoom.us/meeting/regi...,"Apr 21, 2021 05:30 PM Eastern Time (US and Can...",A Free Interactive Presentation for World Lang...,1.619051e+09
687,2021-04-21T13:33:02.000Z,Saddam Sayyaleh\n@SaddamSayyaleh\n·\n18h\nIt's...,https://pbs.twimg.com/media/EzgFU3EXMAIPBRI?fo...,https://twitter.com/SaddamSayyaleh/status/1384...,https://us02web.zoom.us/meeting/register/tZAkc...,"Apr 22, 2021 04:00 PM Amman",Youth Engagement during COVID-19 – Experiences...,1.619132e+09


In [43]:
import pandas as pd
edited=pd.read_csv("TwitterApr15_timestamp.csv")

In [45]:
edited.drop(edited[edited["new_updated_timezone"].isna()].index,inplace=True)

In [50]:
print(len(edited))

391


In [48]:
edited['event_title']=edited['event_title'].astype(str)

In [49]:
edited.drop(edited[edited["event_title"]=="None"].index,inplace=True)

In [51]:
edited.to_csv("Twitter15_dropped.csv",index=False)